# Figures and metrics

Evaluation scores on level-1 entities segmentation and classification with joint-labels method

In [1]:
import os, sys
from pathlib import Path

ENV_IS_GOOGLE_COLAB = True if 'google.colab' in str(get_ipython()) else False
os.environ["ENV_IS_GOOGLE_COLAB"] = str(ENV_IS_GOOGLE_COLAB)

if ENV_IS_GOOGLE_COLAB:
    from google.colab import drive
    mountpoint = Path("/content/drive")
    drive.mount(str(mountpoint)) # Mount gdrive to BASE
    base = mountpoint / "MyDrive/article_icdar_2023" # Adapt this to your situation
    sys.path.append(str(base)) # Add BASE to Python Path
    BASE = Path(base).resolve() # Make BASE absolute
    DATASETS =  BASE / "dataset"
else:
    BASE = Path(os.path.dirname(os.path.realpath("__file__"))).resolve() # If not on GColab, BASE will be the directory of this notebook
    DATASETS = Path('/work/stual/dataset_ICDAR').resolve()
    OUT_BASE = Path('/work/stual/res_ICDAR').resolve()

print(sys.path)
print(BASE)
print(DATASETS)
print(OUT_BASE)

['/lrde/home2/stual/stage_DAS/t2_metrics', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/lrde/home2/stual/.venv/python_3_9/lib/python3.10/site-packages']
/lrde/home2/stual/stage_DAS/t2_metrics
/work/stual/dataset_ICDAR
/work/stual/res_ICDAR


## Constants

In [2]:
MAX_TRAINSET_SIZE = 6084

## Tools

In [3]:
import pandas as pd
import re
import os
import json

def compile_metrics(path): 
    rundirs = [f for f in os.listdir(path)]

    df = pd.DataFrame()
    for run_dir in rundirs:
        if 'run' in run_dir:
            run_path = path / run_dir
            nrun = re.search("\d+",run_dir)[0]

            files = [f for f in os.listdir(run_path) if "test_" in f and 'json' in f]
            sizes = [int(re.search("\d+",f)[0]) for f in files]

            for file, size in zip(files,sizes):
                file_path = run_path / file
                dftmp = pd.read_json(file_path, typ='series')
                dftmp = pd.DataFrame([dftmp])
                dftmp["trainsize"] = size 
                dftmp["run"] = nrun
                dftmp["trainsize_p"] = round(100 * size / MAX_TRAINSET_SIZE, 1)
                df = pd.concat([df, dftmp])

    return df.groupby(["run","trainsize"]).first()

# Load metrics

## Method 1

In [4]:
METRICS_DIR_REF_M1 = OUT_BASE / "method_1/m1-110-experiment_1_metrics"

In [5]:
# Load Camembert IO metrics from metrics jsons
camembert_io_ref_m1 = compile_metrics(METRICS_DIR_REF_M1 / "111-camembert-ner-multihead-io")
camembert_iob2_ref_m1 = compile_metrics(METRICS_DIR_REF_M1 / "112-camembert-ner-multihead-iob2")
prtn_camembert_io_ref_m1 = compile_metrics(METRICS_DIR_REF_M1 / "113-pretrained-camembert-ner-multihead-io")
prtn_camembert_iob2_ref_m1 = compile_metrics(METRICS_DIR_REF_M1 / "114-pretrained-camembert-multihead-iob2")
metrics_raw_m1_ref = pd.concat([camembert_io_ref_m1,camembert_iob2_ref_m1,prtn_camembert_io_ref_m1,prtn_camembert_iob2_ref_m1], keys=["CmBERT IO", "CmBERT IOB2", "CmBERT+ptrn IO", "CmBERT+ptrn IOB2"],names=["Test"])
metrics_raw_m1_ref.head()

eval_precision-all  eval_recall-all  eval_f1-all  \
Test      run trainsize                                                     
CmBERT IO 1   6084                 0.972824         0.965262     0.969028   
          2   6084                 0.967221         0.960525     0.963861   
          3   6084                 0.966561         0.961982     0.964266   
          4   6084                 0.966630         0.957002     0.961792   
          5   6084                 0.967119         0.961011     0.964055   

                         eval_accuracy-all  eval_precision-l1  eval_recall-l1  \
Test      run trainsize                                                         
CmBERT IO 1   6084                0.973220           0.964090        0.969397   
          2   6084                0.962590           0.956407        0.961251   
          3   6084                0.968731           0.959709        0.959709   
          4   6084                0.965893           0.957480        0.956847   
          5   6084                0.967105           0.955071        0.964113   

                         eval_f1-l1  eval_accuracy-l1  eval_precision-l2  \
Test      run trainsize                                                    
CmBERT IO 1   6084         0.966736          0.970846           0.983898   
          2   6084         0.958823          0.951032           0.980892   
          3   6084         0.959709          0.962590           0.975082   
          4   6084         0.957163          0.957224           0.978128   
          5   6084         0.959571          0.961094           0.982481   

                         eval_recall-l2  ...  \
Test      run trainsize                  ...   
CmBERT IO 1   6084             0.960173  ...   
          2   6084             0.959632  ...   
          3   6084             0.964779  ...   
          4   6084             0.957193  ...   
          5   6084             0.957193  ...   

                                                               eval_ACT_L1  \
Test      run trainsize                                                      
CmBERT IO 1   6084       {'precision': 0.957364341085271, 'recall': 0.9...   
          2   6084       {'precision': 0.9659318637274541, 'recall': 0....   
          3   6084       {'precision': 0.9608433734939751, 'recall': 0....   
          4   6084       {'precision': 0.974200206398348, 'recall': 0.9...   
          5   6084       {'precision': 0.9299905392620621, 'recall': 0....   

                                                               eval_ACT_L2  \
Test      run trainsize                                                      
CmBERT IO 1   6084       {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, '...   
          2   6084       {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, '...   
          3   6084       {'precision': 0.28947368421052605, 'recall': 0...   
          4   6084       {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, '...   
          5   6084       {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, '...   

                                                                 eval_DESC  \
Test      run trainsize                                                      
CmBERT IO 1   6084       {'precision': 0.531914893617021, 'recall': 0.6...   
          2   6084       {'precision': 0.35042735042735, 'recall': 0.51...   
          3   6084       {'precision': 0.518867924528301, 'recall': 0.6...   
          4   6084       {'precision': 0.398496240601503, 'recall': 0.6...   
          5   6084       {'precision': 0.39473684210526305, 'recall': 0...   

                                                               eval_TITREH  \
Test      run trainsize                                                      
CmBERT IO 1   6084       {'precision': 0.9772727272727271, 'recall': 1....   
          2   6084       {'precision': 0.976190476190476, 'recall': 0.9...   
          3   6084       {'precision': 0.9772727272727271, 'recall': 1....   
          4   60

In [6]:
METRICS_DIR_OCR_M1 = OUT_BASE / "method_1/m1-120-experiment_2_metrics"

In [7]:
# Load Camembert IO metrics from metrics jsons
camembert_io_ocr_m1 = compile_metrics(METRICS_DIR_OCR_M1 / "121-camembert-ner-multihead-io")
camembert_iob2_ocr_m1 = compile_metrics(METRICS_DIR_OCR_M1 / "122-camembert-ner-multihead-iob2")
prtn_camembert_io_ocr_m1 = compile_metrics(METRICS_DIR_OCR_M1 / "123-pretrained-camembert-ner-multihead-io")
prtn_camembert_iob2_ocr_m1 = compile_metrics(METRICS_DIR_OCR_M1 / "124-pretrained-camembert-multihead-iob2")
metrics_raw_m1_ocr = pd.concat([camembert_io_ocr_m1,camembert_iob2_ocr_m1,prtn_camembert_io_ocr_m1,prtn_camembert_iob2_ocr_m1], keys=["CmBERT IO", "CmBERT IOB2", "CmBERT+ptrn IO", "CmBERT+ptrn IOB2"],names=["Test"])
metrics_raw_m1_ocr.head()

eval_precision-all  eval_recall-all  eval_f1-all  \
Test      run trainsize                                                     
CmBERT IO 1   6084                 0.938887         0.940598     0.939741   
          2   6084                 0.934635         0.936224     0.935429   
          3   6084                 0.932882         0.937075     0.934974   
          4   6084                 0.937803         0.939626     0.938714   
          5   6084                 0.939239         0.938897     0.939068   

                         eval_accuracy-all  eval_precision-l1  eval_recall-l1  \
Test      run trainsize                                                         
CmBERT IO 1   6084                0.960667           0.927178        0.941876   
          2   6084                0.958684           0.918668        0.935051   
          3   6084                0.958605           0.920779        0.936592   
          4   6084                0.960067           0.924446        0.937472   
          5   6084                0.957509           0.927998        0.942096   

                         eval_f1-l1  eval_accuracy-l1  eval_precision-l2  \
Test      run trainsize                                                    
CmBERT IO 1   6084         0.934469          0.955108           0.953757   
          2   6084         0.926787          0.951663           0.955010   
          3   6084         0.928618          0.949157           0.948205   
          4   6084         0.930914          0.951558           0.954695   
          5   6084         0.934994          0.951976           0.953566   

                         eval_recall-l2  ...  \
Test      run trainsize                  ...   
CmBERT IO 1   6084             0.939024  ...   
          2   6084             0.937669  ...   
          3   6084             0.937669  ...   
          4   6084             0.942276  ...   
          5   6084             0.934959  ...   

                                                               eval_ACT_L1  \
Test      run trainsize                                                      
CmBERT IO 1   6084       {'precision': 0.9277456647398841, 'recall': 0....   
          2   6084       {'precision': 0.9007490636704121, 'recall': 0....   
          3   6084       {'precision': 0.9256756756756751, 'recall': 0....   
          4   6084       {'precision': 0.9141791044776121, 'recall': 0....   
          5   6084       {'precision': 0.917613636363636, 'recall': 0.9...   

                                                               eval_ACT_L2  \
Test      run trainsize                                                      
CmBERT IO 1   6084       {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, '...   
          2   6084       {'precision': 0.642857142857142, 'recall': 0.1...   
          3   6084       {'precision': 0.33333333333333304, 'recall': 0...   
          4   6084       {'precision': 0.48717948717948706, 'recall': 0...   
          5   6084       {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, '...   

                                                                 eval_DESC  \
Test      run trainsize                                                      
CmBERT IO 1   6084       {'precision': 0.47272727272727205, 'recall': 0...   
          2   6084       {'precision': 0.49367088607594906, 'recall': 0...   
          3   6084       {'precision': 0.37391304347826004, 'recall': 0...   
          4   6084       {'precision': 0.41428571428571404, 'recall': 0...   
          5   6084       {'precision': 0.556818181818181, 'recall': 0.6...   

                                                               eval_TITREH  \
Test      run trainsize                                                      
CmBERT IO 1   6084       {'precision': 0.69047619047619, 'recall': 0.67...   
          2   6084       {'precision': 0.8285714285714281, 'recall': 0....   
          3   6084       {'precision': 0.622222222222222, 'recall': 0.6...   
          4   60

## Methods 2 and 3

In [8]:
def loadM2M3IOMetrics(path,model_name,res,keys):
    if os.path.exists(f"{path}/run_2"):
        df = compile_metrics(path)
        df["eval_precision-l1l2"] = df["eval_precision"]
        df["eval_recall-l1l2"] = df["eval_recall"]
        df["eval_f1-l1l2"] = df["eval_f1"]
        df["eval_accuracy-l1l2"] = df["eval_accuracy"]
        res.append(df)
        keys.append(model_name)
    
def loadM2M3IOB2Metrics(path,model_name,res,keys):
    if os.path.exists(f"{path}/run_2"):
        df = compile_metrics(path)
        res.append(df)
        keys.append(model_name)

#### Method 2

In [9]:
METRICS_DIR_REF_M2 = OUT_BASE / "method_2/m2-210-experiment_1_metrics"

In [10]:
res_m2_ref = []
keys_m2_ref = []

camembert_ner_io_ref_m2 = METRICS_DIR_REF_M2 / "211-camembert-ner-joint-labelling-io"
loadM2M3IOMetrics(camembert_ner_io_ref_m2,"CmBERT IO",res_m2_ref,keys_m2_ref)

camembert_ner_iob2_ref_m2 = METRICS_DIR_REF_M2 / "212-camembert-ner-joint-labelling-iob2"
loadM2M3IOB2Metrics(camembert_ner_iob2_ref_m2,"CmBERT IOB2",res_m2_ref,keys_m2_ref)

ptrn_camembert_ner_io_ref_m2 = METRICS_DIR_REF_M2 / "213-pretrained-camembert-ner-joint-labelling-io"
loadM2M3IOMetrics(ptrn_camembert_ner_io_ref_m2,"CmBERT+ptrn IO",res_m2_ref,keys_m2_ref)

ptrn_camembert_ner_iob2_ref_m2 = METRICS_DIR_REF_M2 / "214-pretrained-camembert-ner-joint-labelling-iob2"
loadM2M3IOB2Metrics(ptrn_camembert_ner_iob2_ref_m2,"CmBERT+ptrn IOB2",res_m2_ref,keys_m2_ref)

In [11]:
metrics_raw_m2_ref = pd.concat(res_m2_ref, keys=keys_m2_ref,names=["Test"])
metrics_raw_m2_ref.head()

eval_loss  eval_precision  eval_recall   eval_f1  \
Test      run trainsize                                                     
CmBERT IO 1   6084        0.191235        0.959145     0.967932  0.963519   
          2   6084        0.219643        0.959671     0.969620  0.964620   
          3   6084        0.216868        0.960798     0.969138  0.964950   
          4   6084        0.191439        0.955330     0.953948  0.954639   
          5   6084        0.163849        0.957006     0.963351  0.960168   

                         eval_accuracy  eval_precision-l1  eval_recall-l1  \
Test      run trainsize                                                     
CmBERT IO 1   6084            0.942002           0.957034        0.966094   
          2   6084            0.948091           0.957345        0.968516   
          3   6084            0.944221           0.961530        0.968516   
          4   6084            0.932921           0.922617        0.950242   
          5   6084            0.943137           0.948100        0.961251   

                         eval_f1-l1  eval_accuracy-l1  eval_precision-l2  ...  \
Test      run trainsize                                                   ...   
CmBERT IO 1   6084         0.961543          0.965067           0.961497  ...   
          2   6084         0.962898          0.969350           0.963340  ...   
          3   6084         0.965010          0.966254           0.965674  ...   
          4   6084         0.936226          0.938029           0.981205  ...   
          5   6084         0.954630          0.961094           0.961652  ...   

                         eval_runtime  eval_samples_per_second  \
Test      run trainsize                                          
CmBERT IO 1   6084             9.0450                  186.291   
          2   6084             8.7241                  193.143   
          3   6084            10.7733                  156.405   
          4   6084            10.5994                  158.971   
          5   6084             9.3141                  180.909   

                         eval_steps_per_second  epoch  trainsize_p  \
Test      run trainsize                                              
CmBERT IO 1   6084                      11.719   6.30        100.0   
          2   6084                      12.150  10.24        100.0   
          3   6084                       9.839   9.45        100.0   
          4   6084                      10.001   2.89        100.0   
          5   6084                      11.381   3.94        100.0   

                                                                eval_TITRE  \
Test      run trainsize                                                      
CmBERT IO 1   6084                                                    None   
          2   6084       {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, '...   
          3   6084       {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, '...   
          4   6084                                                    None   
          5   6084                                                    None   

                         eval_precision-l1l2  eval_recall-l1l2  eval_f1-l1l2  \
Test      run trainsize                                                        
CmBERT IO 1   6084                  0.959145          0.967932      0.963519   
          2   6084                  0.959671          0.969620      0.964620   
          3   6084                  0.960798          0.969138      0.964950   
          4   6084                  0.955330          0.953948      0.954639   
          5   6084                  0.957006          0.963351      0.960168   

                         eval_accuracy-l1l2  
Test      run trainsize                      
CmBERT IO 1   6084                 0.942002  
          2   6084                 0.948091  
          3   6084                 0.944221  
          4   6084                 0.932921  
          5   6084                 0.94

In [12]:
METRICS_DIR_OCR_M2 = OUT_BASE / "method_2/m2-220-experiment_2_metrics"

In [13]:
res_m2_ocr = []
keys_m2_ocr = []

camembert_ner_io_ocr_m2 = METRICS_DIR_OCR_M2 / "221-camembert-ner-joint-labelling-io"
loadM2M3IOMetrics(camembert_ner_io_ocr_m2,"CmBERT IO",res_m2_ocr,keys_m2_ocr)

camembert_ner_iob2_ocr_m2 = METRICS_DIR_OCR_M2 / "222-camembert-ner-joint-labelling-iob2"
loadM2M3IOB2Metrics(camembert_ner_iob2_ocr_m2,"CmBERT IOB2",res_m2_ocr,keys_m2_ocr)

ptrn_camembert_ner_io_ocr_m2 = METRICS_DIR_OCR_M2 / "223-pretrained-camembert-ner-joint-labelling-io"
loadM2M3IOMetrics(ptrn_camembert_ner_io_ocr_m2,"CmBERT+ptrn IO",res_m2_ocr,keys_m2_ocr)

ptrn_camembert_ner_iob2_ocr_m2 = METRICS_DIR_OCR_M2 / "224-pretrained-camembert-ner-joint-labelling-iob2"
loadM2M3IOB2Metrics(ptrn_camembert_ner_iob2_ocr_m2,"CmBERT+ptrn IOB2",res_m2_ocr,keys_m2_ocr)

In [14]:
metrics_raw_m2_ocr = pd.concat(res_m2_ocr, keys=keys_m2_ocr,names=["Test"])
metrics_raw_m2_ocr.head()

eval_loss  eval_precision  eval_recall   eval_f1  \
Test      run trainsize                                                     
CmBERT IO 1   6084        0.291394        0.940070     0.948253  0.944143   
          2   6084        0.254665        0.942836     0.945230  0.944032   
          3   6084        0.209478        0.940993     0.946681  0.943828   
          4   6084        0.240886        0.924360     0.938218  0.931237   
          5   6084        0.227982        0.938368     0.948011  0.943165   

                         eval_accuracy  eval_precision-l1  eval_recall-l1  \
Test      run trainsize                                                     
CmBERT IO 1   6084            0.937151           0.932435        0.944958   
          2   6084            0.928642           0.926398        0.944958   
          3   6084            0.935272           0.924157        0.941656   
          4   6084            0.923996           0.908330        0.931528   
          5   6084            0.934645           0.931735        0.949582   

                         eval_f1-l1  eval_accuracy-l1  eval_precision-l2  ...  \
Test      run trainsize                                                   ...   
CmBERT IO 1   6084         0.938655          0.961215           0.944160  ...   
          2   6084         0.935586          0.950775           0.944324  ...   
          3   6084         0.932824          0.955943           0.946341  ...   
          4   6084         0.919783          0.950827           0.934964  ...   
          5   6084         0.940574          0.959127           0.941763  ...   

                         eval_runtime  eval_samples_per_second  \
Test      run trainsize                                          
CmBERT IO 1   6084            10.5194                  160.181   
          2   6084            10.5746                  159.345   
          3   6084             8.9153                  189.000   
          4   6084             9.2637                  181.893   
          5   6084            10.3430                  162.912   

                         eval_steps_per_second  epoch  trainsize_p  \
Test      run trainsize                                              
CmBERT IO 1   6084                      10.077   8.92        100.0   
          2   6084                      10.024   4.72        100.0   
          3   6084                      11.890   4.46        100.0   
          4   6084                      11.443   3.94        100.0   
          5   6084                      10.248   6.56        100.0   

                                                                eval_TITRE  \
Test      run trainsize                                                      
CmBERT IO 1   6084       {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, '...   
          2   6084                                                    None   
          3   6084                                                    None   
          4   6084                                                    None   
          5   6084                                                    None   

                         eval_precision-l1l2  eval_recall-l1l2  eval_f1-l1l2  \
Test      run trainsize                                                        
CmBERT IO 1   6084                  0.940070          0.948253      0.944143   
          2   6084                  0.942836          0.945230      0.944032   
          3   6084                  0.940993          0.946681      0.943828   
          4   6084                  0.924360          0.938218      0.931237   
          5   6084                  0.938368          0.948011      0.943165   

                         eval_accuracy-l1l2  
Test      run trainsize                      
CmBERT IO 1   6084                 0.937151  
          2   6084                 0.928642  
          3   6084                 0.935272  
          4   6084                 0.923996  
          5   6084                 0.93

#### Method 3

In [15]:
METRICS_DIR_REF_M3 = OUT_BASE / "method_3/m3-310-experiment_1_metrics"

In [16]:
res_m3_ref = []
keys_m3_ref = []

camembert_ner_io_ref_m3 = METRICS_DIR_REF_M3 / "311-camembert-ner-hierarchical-loss-io"
loadM2M3IOMetrics(camembert_ner_io_ref_m3,"CmBERT IO",res_m3_ref,keys_m3_ref)

camembert_ner_iob2_ref_m3 = METRICS_DIR_REF_M3 / "312-camembert-ner-hierarchical-loss-iob2"
loadM2M3IOB2Metrics(camembert_ner_iob2_ref_m3,"CmBERT IOB2",res_m3_ref,keys_m3_ref)

ptrn_camembert_ner_io_ref_m3 = METRICS_DIR_REF_M3 / "313-pretrained-camembert-ner-hierarchical-loss-io"
loadM2M3IOMetrics(ptrn_camembert_ner_io_ref_m3,"CmBERT+ptrn IO",res_m3_ref,keys_m3_ref)

ptrn_camembert_ner_iob2_ref_m3 = METRICS_DIR_REF_M3 / "314-pretrained-camembert-ner-hierarchical-loss-iob2"
loadM2M3IOB2Metrics(ptrn_camembert_ner_iob2_ref_m3,"CmBERT+ptrn IOB2",res_m3_ref,keys_m3_ref)

In [17]:
metrics_raw_m3_ref = pd.concat(res_m3_ref, keys=keys_m3_ref,names=["Test"])
metrics_raw_m3_ref.head()

eval_loss  eval_precision  eval_recall   eval_f1  \
Test      run trainsize                                                     
CmBERT IO 1   6084        0.165812        0.958842     0.966124  0.962469   
          2   6084        0.216428        0.956278     0.965039  0.960638   
          3   6084        0.173049        0.956678     0.961061  0.958865   
          4   6084        0.274444        0.960791     0.966004  0.963390   
          5   6084        0.154278        0.956179     0.965401  0.960768   

                         eval_accuracy  eval_precision-l1  eval_recall-l1  \
Test      run trainsize                                                     
CmBERT IO 1   6084            0.947626           0.940484        0.967195   
          2   6084            0.941538           0.952007        0.960810   
          3   6084            0.934469           0.950437        0.958388   
          4   6084            0.942621           0.957860        0.965874   
          5   6084            0.940454           0.958388        0.963452   

                         eval_f1-l1  eval_accuracy-l1  eval_precision-l2  ...  \
Test      run trainsize                                                   ...   
CmBERT IO 1   6084         0.953652          0.968215           0.967054  ...   
          2   6084         0.956388          0.960578           0.960779  ...   
          3   6084         0.954396          0.958514           0.963568  ...   
          4   6084         0.961850          0.962487           0.969534  ...   
          5   6084         0.960913          0.964241           0.961642  ...   

                                                                eval_TITRE  \
Test      run trainsize                                                      
CmBERT IO 1   6084                                                    None   
          2   6084                                                    None   
          3   6084                                                    None   
          4   6084       {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, '...   
          5   6084                                                    None   

                         eval_runtime  eval_samples_per_second  \
Test      run trainsize                                          
CmBERT IO 1   6084            21.1232                   79.770   
          2   6084            25.0354                   67.305   
          3   6084            26.0704                   64.633   
          4   6084            15.4560                  109.019   
          5   6084            56.4387                   29.855   

                         eval_steps_per_second  epoch  trainsize_p  \
Test      run trainsize                                              
CmBERT IO 1   6084                       5.018   3.67        100.0   
          2   6084                       4.234   6.04        100.0   
          3   6084                       4.066   3.94        100.0   
          4   6084                       6.858  11.81        100.0   
          5   6084                       1.878   4.46        100.0   

                         eval_precision-l1l2  eval_recall-l1l2  eval_f1-l1l2  \
Test      run trainsize                                                        
CmBERT IO 1   6084                  0.958842          0.966124      0.962469   
          2   6084                  0.956278          0.965039      0.960638   
          3   6084                  0.956678          0.961061      0.958865   
          4   6084                  0.960791          0.966004      0.963390   
          5   6084                  0.956179          0.965401      0.960768   

                         eval_accuracy-l1l2  
Test      run trainsize                      
CmBERT IO 1   6084                 0.947626  
          2   6084                 0.941538  
          3   6084                 0.934469  
          4   6084                 0.942621  
          5   6084                 0.94

In [18]:
METRICS_DIR_OCR_M3 = OUT_BASE / "method_3/m3-320-experiment_2_metrics"

In [19]:
res_m3_ocr = []
keys_m3_ocr = []

camembert_ner_io_ocr_m3 = METRICS_DIR_OCR_M3 / "321-camembert-ner-hierarchical-loss-io"
loadM2M3IOMetrics(camembert_ner_io_ocr_m3,"CmBERT IO",res_m3_ocr,keys_m3_ocr)

camembert_ner_iob2_ocr_m3 = METRICS_DIR_OCR_M3 / "322-camembert-ner-hierarchical-loss-iob2"
loadM2M3IOB2Metrics(camembert_ner_iob2_ocr_m3,"CmBERT IOB2",res_m3_ocr,keys_m3_ocr)

ptrn_camembert_ner_io_ocr_m3 = METRICS_DIR_OCR_M3 / "323-pretrained-camembert-ner-hierarchical-loss-io"
loadM2M3IOMetrics(ptrn_camembert_ner_io_ocr_m3,"CmBERT+ptrn IO",res_m3_ocr,keys_m3_ocr)

ptrn_camembert_ner_iob2_ocr_m3 = METRICS_DIR_OCR_M3 / "324-pretrained-camembert-ner-hierarchical-loss-iob2"
loadM2M3IOB2Metrics(ptrn_camembert_ner_iob2_ocr_m3,"CmBERT+ptrn IOB2",res_m3_ocr,keys_m3_ocr)

In [20]:
metrics_raw_m3_ocr = pd.concat(res_m3_ocr, keys=keys_m3_ocr,names=["Test"])
metrics_raw_m3_ocr.head()

eval_loss  eval_precision  eval_recall   eval_f1  \
Test      run trainsize                                                     
CmBERT IO 1   6084        0.256559        0.947254     0.944505  0.945877   
          2   6084        0.223901        0.930058     0.945351  0.937642   
          3   6084        0.241124        0.943616     0.948978  0.946290   
          4   6084        0.264271        0.941240     0.948978  0.945093   
          5   6084        0.242043        0.940459     0.951034  0.945717   

                         eval_accuracy  eval_precision-l1  eval_recall-l1  \
Test      run trainsize                                                     
CmBERT IO 1   6084            0.933810           0.928977        0.935931   
          2   6084            0.931148           0.913155        0.939894   
          3   6084            0.937621           0.935105        0.945399   
          4   6084            0.932296           0.933435        0.941656   
          5   6084            0.933967           0.933493        0.945619   

                         eval_f1-l1  eval_accuracy-l1  eval_precision-l2  ...  \
Test      run trainsize                                                   ...   
CmBERT IO 1   6084         0.932441          0.956361           0.955930  ...   
          2   6084         0.926332          0.951245           0.936932  ...   
          3   6084         0.940223          0.958866           0.946150  ...   
          4   6084         0.937527          0.954690           0.945058  ...   
          5   6084         0.939517          0.957666           0.943280  ...   

                                                                eval_TITRE  \
Test      run trainsize                                                      
CmBERT IO 1   6084                                                    None   
          2   6084                                                    None   
          3   6084                                                    None   
          4   6084       {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, '...   
          5   6084                                                    None   

                         eval_runtime  eval_samples_per_second  \
Test      run trainsize                                          
CmBERT IO 1   6084             9.0527                  186.133   
          2   6084             9.2894                  181.390   
          3   6084             8.6041                  195.838   
          4   6084             9.4472                  178.360   
          5   6084             8.8687                  189.995   

                         eval_steps_per_second  epoch  trainsize_p  \
Test      run trainsize                                              
CmBERT IO 1   6084                      11.709   5.25        100.0   
          2   6084                      11.411   7.09        100.0   
          3   6084                      12.320   7.09        100.0   
          4   6084                      11.220   7.61        100.0   
          5   6084                      11.952   6.56        100.0   

                         eval_precision-l1l2  eval_recall-l1l2  eval_f1-l1l2  \
Test      run trainsize                                                        
CmBERT IO 1   6084                  0.947254          0.944505      0.945877   
          2   6084                  0.930058          0.945351      0.937642   
          3   6084                  0.943616          0.948978      0.946290   
          4   6084                  0.941240          0.948978      0.945093   
          5   6084                  0.940459          0.951034      0.945717   

                         eval_accuracy-l1l2  
Test      run trainsize                      
CmBERT IO 1   6084                 0.933810  
          2   6084                 0.931148  
          3   6084                 0.937621  
          4   6084                 0.932296  
          5   6084                 0.93

In [21]:
ref = [metrics_raw_m1_ref,metrics_raw_m2_ref,metrics_raw_m3_ref]
keys_ref = ["M1","M2","M3"]
metrics_raw_ref = pd.concat(ref, keys=keys_ref,names=["Approach"])

ocr = [metrics_raw_m1_ocr, metrics_raw_m2_ocr,metrics_raw_m3_ocr]
keys_ocr = ["M1","M2","M3"]
metrics_raw_ocr = pd.concat(ocr, keys=keys_ocr,names=["Approach"])

datasets = [metrics_raw_ref,metrics_raw_ocr]
keys = ["Reference","OCR"]
metrics_raw = pd.concat(datasets, keys=keys,names=["Dataset"])
metrics_raw

eval_precision-all  \
Dataset   Approach Test             run trainsize                       
Reference M1       CmBERT IO        1   6084                 0.972824   
                                    2   6084                 0.967221   
                                    3   6084                 0.966561   
                                    4   6084                 0.966630   
                                    5   6084                 0.967119   
...                                                               ...   
OCR       M3       CmBERT+ptrn IOB2 1   6084                 0.924840   
                                    2   6084                 0.925077   
                                    3   6084                 0.929074   
                                    4   6084                 0.927008   
                                    5   6084                 0.924537   

                                                   eval_recall-all  \
Dataset   Approach Test             run trainsize                    
Reference M1       CmBERT IO        1   6084              0.965262   
                                    2   6084              0.960525   
                                    3   6084              0.961982   
                                    4   6084              0.957002   
                                    5   6084              0.961011   
...                                                            ...   
OCR       M3       CmBERT+ptrn IOB2 1   6084              0.945956   
                                    2   6084              0.946199   
                                    3   6084              0.946563   
                                    4   6084              0.951664   
                                    5   6084              0.946320   

                                                   eval_f1-all  \
Dataset   Approach Test             run trainsize                
Reference M1       CmBERT IO        1   6084          0.969028   
                                    2   6084          0.963861   
                                    3   6084          0.964266   
                                    4   6084          0.961792   
                                    5   6084          0.964055   
...                                                        ...   
OCR       M3       CmBERT+ptrn IOB2 1   6084          0.935279   
                                    2   6084          0.935519   
                                    3   6084          0.937737   
                                    4   6084          0.939174   
                                    5   6084          0.935302   

                                                   eval_accuracy-all  \
Dataset   Approach Test             run trainsize                      
Reference M1       CmBERT IO        1   6084                0.973220   
                                    2   6084                0.962590   
                                    3   6084                0.968731   
                                    4   6084                0.965893   
                                    5   6084                0.967105   
...                                                              ...   
OCR       M3       CmBERT+ptrn IOB2 1   6084                0.954038   
                                    2   6084                0.951532   
                                    3   6084                0.952341   
                                    4   6084                0.953907   
                                    5   6084                0.954899   

                                                   eval_precision-l1  \
Dataset   Approach Test             run trainsize                      
Reference M1       CmBERT IO        1   6084                0.964090   
                                    2   6084                0.956407   
                                    3   6084                0.959709   
                                  

## 231.1 Build the averaged table


In [22]:
eval_ = []
for elem in metrics_raw.columns:
    if 'eval_p' in elem or 'eval_re' in elem or 'eval_f' in elem or 'eval_ac' in elem:
        eval_.append(elem)

In [23]:
metrics_raw = metrics_raw.groupby(level=(0,1,2)).mean()
metrics_raw[eval_] = metrics_raw[eval_].multiply(100., axis=1)
metrics_raw 

eval_precision-all  eval_recall-all  \
Dataset   Approach Test                                                    
Reference M1       CmBERT IO                  96.807098        96.115632   
                   CmBERT IOB2                95.967198        96.470731   
                   CmBERT+ptrn IO             96.567890        96.674359   
                   CmBERT+ptrn IOB2           95.520199        96.512023   
          M2       CmBERT IO                  95.695905        96.686506   
                   CmBERT IOB2                95.437932        96.533884   
                   CmBERT+ptrn IO             95.594920        96.995020   
                   CmBERT+ptrn IOB2           95.250306        96.920087   
          M3       CmBERT IO                  95.749072        96.764241   
                   CmBERT IOB2                95.607198        96.633471   
                   CmBERT+ptrn IO             95.191700        96.509170   
                   CmBERT+ptrn IOB2           95.563402        97.121691   
OCR       M1       CmBERT IO                  93.668921        93.848397   
                   CmBERT IOB2                93.004364        94.010202   
                   CmBERT+ptrn IO             93.960445        94.681730   
                   CmBERT+ptrn IOB2           93.570140        94.607724   
          M2       CmBERT IO                  93.247057        94.378037   
                   CmBERT IOB2                92.896867        94.663590   
                   CmBERT+ptrn IO             93.082507        94.696307   
                   CmBERT+ptrn IOB2           92.614194        94.857906   
          M3       CmBERT IO                  93.625178        94.601555   
                   CmBERT IOB2                92.743344        94.308963   
                   CmBERT+ptrn IO             93.396062        94.839650   
                   CmBERT+ptrn IOB2           92.610722        94.734030   

                                     eval_f1-all  eval_accuracy-all  \
Dataset   Approach Test                                               
Reference M1       CmBERT IO           96.460054          96.750774   
                   CmBERT IOB2         96.217547          96.542828   
                   CmBERT+ptrn IO      96.620776          96.987616   
                   CmBERT+ptrn IOB2    96.012234          96.654799   
          M2       CmBERT IO           96.188573          96.427761   
                   CmBERT IOB2         95.982694          96.369453   
                   CmBERT+ptrn IO      96.289239          96.640867   
                   CmBERT+ptrn IOB2    96.077814          96.529928   
          M3       CmBERT IO           96.253651          96.424149   
                   CmBERT IOB2         96.117493          96.475232   
                   CmBERT+ptrn IO      95.845413          96.237358   
                   CmBERT+ptrn IOB2    96.336005          96.707430   
OCR       M1       CmBERT IO           93.758518          95.910633   
                   CmBERT IOB2         93.503830          95.495119   
                   CmBERT+ptrn IO      94.318797          96.115780   
                   CmBERT+ptrn IOB2    94.085092          95.661638   
          M2       CmBERT IO           93.808923          95.785875   
                   CmBERT IOB2         93.771105          95.350525   
                   CmBERT+ptrn IO      93.882361          95.725322   
                   CmBERT+ptrn IOB2    93.722318          95.388109   
          M3       CmBERT IO           94.109835          95.913243   
                   CmBERT IOB2         93.518969          95.267004   
                   CmBERT+ptrn IO      94.111771          95.722712   
                   CmBERT+ptrn IOB2    93.660205          95.334343   

                                     eval_precision-l1  eval_recall-l1  \
Dataset   Approach Test                                                  
Reference M1       CmBERT IO                 95.855151       96.2

In [24]:
averaged = metrics_raw.copy()
averaged = averaged[["eval_f1-all","eval_f1-l1l2","eval_f1-l1","eval_f1-l2","eval_f1","eval_f1-das"]]

# Set pretty names
averaged.index.names = ['Dataset','Approach',"Model and tags"]
averaged.rename(columns={"eval_f1-all":"All",
                        "eval_f1-l1":"L1",
                        "eval_f1-l2":"L2",
                         "eval_f1-l1l2":"L1&L2",
                         "eval_f1":"P-L1+P-L2",
                         "eval_f1-das":"Flat",
                         }, errors="raise", inplace=True)
#averaged.rename(mapper={"camembert_io_ref": "CmBERT IO","camembert_iob2_ref": "CmBERT IOB2","prtn_camembert_io_ref": "Ptrn CmBERT IO","prtn_camembert_iob2_ref": "Ptrn CmBERT IOB2"}, errors="ignore", inplace=True, axis=0)
averaged

All      L1&L2         L1  \
Dataset   Approach Model and tags                                      
Reference M1       CmBERT IO         96.460054  95.699654  96.040045   
                   CmBERT IOB2       96.217547  95.614767  95.789203   
                   CmBERT+ptrn IO    96.620776  95.918409  96.317256   
                   CmBERT+ptrn IOB2  96.012234  95.212618  95.528058   
          M2       CmBERT IO         96.188573  96.157909  95.606135   
                   CmBERT IOB2       95.982694  96.030419  95.291741   
                   CmBERT+ptrn IO    96.289239  96.125104  95.970882   
                   CmBERT+ptrn IOB2  96.077814  96.035353  95.846368   
          M3       CmBERT IO         96.253651  96.122610  95.744014   
                   CmBERT IOB2       96.117493  96.139536  95.621790   
                   CmBERT+ptrn IO    95.845413  95.752708  95.188101   
                   CmBERT+ptrn IOB2  96.336005  96.283451  96.024833   
OCR       M1       CmBERT IO         93.758518  93.355103  93.115639   
                   CmBERT IOB2       93.503830  92.922154  93.109544   
                   CmBERT+ptrn IO    94.318797  93.835619  94.133931   
                   CmBERT+ptrn IOB2  94.085092  93.496475  93.743182   
          M2       CmBERT IO         93.808923  94.128114  93.348428   
                   CmBERT IOB2       93.771105  94.161059  93.195417   
                   CmBERT+ptrn IO    93.882361  94.089300  93.437655   
                   CmBERT+ptrn IOB2  93.722318  94.119129  93.064214   
          M3       CmBERT IO         94.109835  94.412390  93.520808   
                   CmBERT IOB2       93.518969  93.907747  92.901022   
                   CmBERT+ptrn IO    94.111771  94.398637  93.579720   
                   CmBERT+ptrn IOB2  93.660205  94.038940  93.075170   

                                            L2  P-L1+P-L2       Flat  
Dataset   Approach Model and tags                                     
Reference M1       CmBERT IO         96.983179  95.699654  97.042905  
                   CmBERT IOB2       96.751012  95.663863  96.135266  
                   CmBERT+ptrn IO    96.999829  95.918409  97.268102  
                   CmBERT+ptrn IOB2  96.612880  95.331939  95.250599  
          M2       CmBERT IO         96.914893  96.157909  96.703881  
                   CmBERT IOB2       96.839731  96.035288  96.737652  
                   CmBERT+ptrn IO    96.682303  96.125104  96.877739  
                   CmBERT+ptrn IOB2  96.363276  96.068198  96.912063  
          M3       CmBERT IO         96.882258  96.122610  96.736881  
                   CmBERT IOB2       96.729561  96.144481  96.802549  
                   CmBERT+ptrn IO    96.663223  95.752708  96.411974  
                   CmBERT+ptrn IOB2  96.720151  96.254617  97.017190  
OCR       M1       CmBERT IO         94.562345  93.355103  94.158076  
                   CmBERT IOB2       93.993053  93.127077  92.710741  
                   CmBERT+ptrn IO    94.548789  93.835619  94.419470  
                   CmBERT+ptrn IOB2  94.510817  93.708233  94.545170  
          M2       CmBERT IO         94.380526  94.128114  94.534562  
                   CmBERT IOB2       94.486534  94.281452  94.690020  
                   CmBERT+ptrn IO    94.432368  94.089300  94.612859  
                   CmBERT+ptrn IOB2  94.535459  94.189938  94.847359  
          M3       CmBERT IO         94.838207  94.412390  94.773225  
                   CmBERT IOB2       94.285655  93.990849  94.583356  
                   CmBERT+ptrn IO    94.769611  94.398637  94.914480  
                   CmBERT+ptrn IOB2  94.385660  94.189544  94.808493

In [25]:
averaged_ref = averaged.loc[['Reference']]
averaged_ref = averaged_ref.reset_index(0)
del averaged_ref["Dataset"]
latex_table = averaged_ref.copy()
caption = "F1 score measured for each approach, pre-trained model and tag format (mean of 5 runs) on the reference dataset."
print(latex_table.to_latex(float_format="%.1f", multirow=True, caption=caption))
latex_table

\begin{table}
\centering
\caption{F1 score measured for each approach, pre-trained model and tag format (mean of 5 runs) on the reference dataset.}
\begin{tabular}{llrrrrrr}
\toprule
   &                  &  All &  L1\&L2 &   L1 &   L2 &  P-L1+P-L2 &  Flat \\
Approach & Model and tags &      &        &      &      &            &       \\
\midrule
\multirow{4}{*}{M1} & CmBERT IO & 96.5 &   95.7 & 96.0 & 97.0 &       95.7 &  97.0 \\
   & CmBERT IOB2 & 96.2 &   95.6 & 95.8 & 96.8 &       95.7 &  96.1 \\
   & CmBERT+ptrn IO & 96.6 &   95.9 & 96.3 & 97.0 &       95.9 &  97.3 \\
   & CmBERT+ptrn IOB2 & 96.0 &   95.2 & 95.5 & 96.6 &       95.3 &  95.3 \\
\cline{1-8}
\multirow{4}{*}{M2} & CmBERT IO & 96.2 &   96.2 & 95.6 & 96.9 &       96.2 &  96.7 \\
   & CmBERT IOB2 & 96.0 &   96.0 & 95.3 & 96.8 &       96.0 &  96.7 \\
   & CmBERT+ptrn IO & 96.3 &   96.1 & 96.0 & 96.7 &       96.1 &  96.9 \\
   & CmBERT+ptrn IOB2 & 96.1 &   96.0 & 95.8 & 96.4 &       96.1 &  96.9 \\
\cline{1-8}
\multirow{4}{

All      L1&L2         L1         L2  \
Approach Model and tags                                                 
M1       CmBERT IO         96.460054  95.699654  96.040045  96.983179   
         CmBERT IOB2       96.217547  95.614767  95.789203  96.751012   
         CmBERT+ptrn IO    96.620776  95.918409  96.317256  96.999829   
         CmBERT+ptrn IOB2  96.012234  95.212618  95.528058  96.612880   
M2       CmBERT IO         96.188573  96.157909  95.606135  96.914893   
         CmBERT IOB2       95.982694  96.030419  95.291741  96.839731   
         CmBERT+ptrn IO    96.289239  96.125104  95.970882  96.682303   
         CmBERT+ptrn IOB2  96.077814  96.035353  95.846368  96.363276   
M3       CmBERT IO         96.253651  96.122610  95.744014  96.882258   
         CmBERT IOB2       96.117493  96.139536  95.621790  96.729561   
         CmBERT+ptrn IO    95.845413  95.752708  95.188101  96.663223   
         CmBERT+ptrn IOB2  96.336005  96.283451  96.024833  96.720151   

                           P-L1+P-L2       Flat  
Approach Model and tags                          
M1       CmBERT IO         95.699654  97.042905  
         CmBERT IOB2       95.663863  96.135266  
         CmBERT+ptrn IO    95.918409  97.268102  
         CmBERT+ptrn IOB2  95.331939  95.250599  
M2       CmBERT IO         96.157909  96.703881  
         CmBERT IOB2       96.035288  96.737652  
         CmBERT+ptrn IO    96.125104  96.877739  
         CmBERT+ptrn IOB2  96.068198  96.912063  
M3       CmBERT IO         96.122610  96.736881  
         CmBERT IOB2       96.144481  96.802549  
         CmBERT+ptrn IO    95.752708  96.411974  
         CmBERT+ptrn IOB2  96.254617  97.017190

In [26]:
averaged_ref = averaged.loc[['OCR']]
averaged_ref = averaged_ref.reset_index(0)
del averaged_ref["Dataset"]
latex_table = averaged_ref.copy()
caption = "F1 score measured for each approach, pre-trained model and tag format (mean of 5 runs) on the noisy dataset."
print(latex_table.to_latex(float_format="%.1f", multirow=True, caption=caption))
latex_table

\begin{table}
\centering
\caption{F1 score measured for each approach, pre-trained model and tag format (mean of 5 runs) on the noisy dataset.}
\begin{tabular}{llrrrrrr}
\toprule
   &                  &  All &  L1\&L2 &   L1 &   L2 &  P-L1+P-L2 &  Flat \\
Approach & Model and tags &      &        &      &      &            &       \\
\midrule
\multirow{4}{*}{M1} & CmBERT IO & 93.8 &   93.4 & 93.1 & 94.6 &       93.4 &  94.2 \\
   & CmBERT IOB2 & 93.5 &   92.9 & 93.1 & 94.0 &       93.1 &  92.7 \\
   & CmBERT+ptrn IO & 94.3 &   93.8 & 94.1 & 94.5 &       93.8 &  94.4 \\
   & CmBERT+ptrn IOB2 & 94.1 &   93.5 & 93.7 & 94.5 &       93.7 &  94.5 \\
\cline{1-8}
\multirow{4}{*}{M2} & CmBERT IO & 93.8 &   94.1 & 93.3 & 94.4 &       94.1 &  94.5 \\
   & CmBERT IOB2 & 93.8 &   94.2 & 93.2 & 94.5 &       94.3 &  94.7 \\
   & CmBERT+ptrn IO & 93.9 &   94.1 & 93.4 & 94.4 &       94.1 &  94.6 \\
   & CmBERT+ptrn IOB2 & 93.7 &   94.1 & 93.1 & 94.5 &       94.2 &  94.8 \\
\cline{1-8}
\multirow{4}{*}{M

All      L1&L2         L1         L2  \
Approach Model and tags                                                 
M1       CmBERT IO         93.758518  93.355103  93.115639  94.562345   
         CmBERT IOB2       93.503830  92.922154  93.109544  93.993053   
         CmBERT+ptrn IO    94.318797  93.835619  94.133931  94.548789   
         CmBERT+ptrn IOB2  94.085092  93.496475  93.743182  94.510817   
M2       CmBERT IO         93.808923  94.128114  93.348428  94.380526   
         CmBERT IOB2       93.771105  94.161059  93.195417  94.486534   
         CmBERT+ptrn IO    93.882361  94.089300  93.437655  94.432368   
         CmBERT+ptrn IOB2  93.722318  94.119129  93.064214  94.535459   
M3       CmBERT IO         94.109835  94.412390  93.520808  94.838207   
         CmBERT IOB2       93.518969  93.907747  92.901022  94.285655   
         CmBERT+ptrn IO    94.111771  94.398637  93.579720  94.769611   
         CmBERT+ptrn IOB2  93.660205  94.038940  93.075170  94.385660   

                           P-L1+P-L2       Flat  
Approach Model and tags                          
M1       CmBERT IO         93.355103  94.158076  
         CmBERT IOB2       93.127077  92.710741  
         CmBERT+ptrn IO    93.835619  94.419470  
         CmBERT+ptrn IOB2  93.708233  94.545170  
M2       CmBERT IO         94.128114  94.534562  
         CmBERT IOB2       94.281452  94.690020  
         CmBERT+ptrn IO    94.089300  94.612859  
         CmBERT+ptrn IOB2  94.189938  94.847359  
M3       CmBERT IO         94.412390  94.773225  
         CmBERT IOB2       93.990849  94.583356  
         CmBERT+ptrn IO    94.398637  94.914480  
         CmBERT+ptrn IOB2  94.189544  94.808493

## 231.2 Create the results table

In [27]:
latex_table = averaged.copy()

caption = "F1 score measured for each approach, pre-trained model and tag format (mean of 5 runs)."
print(latex_table.to_latex(float_format="%.1f", multirow=True, caption=caption))
latex_table

\begin{table}
\centering
\caption{F1 score measured for each approach, pre-trained model and tag format (mean of 5 runs).}
\begin{tabular}{lllrrrrrr}
\toprule
    &    &                  &  All &  L1\&L2 &   L1 &   L2 &  P-L1+P-L2 &  Flat \\
Dataset & Approach & Model and tags &      &        &      &      &            &       \\
\midrule
\multirow{12}{*}{Reference} & \multirow{4}{*}{M1} & CmBERT IO & 96.5 &   95.7 & 96.0 & 97.0 &       95.7 &  97.0 \\
    &    & CmBERT IOB2 & 96.2 &   95.6 & 95.8 & 96.8 &       95.7 &  96.1 \\
    &    & CmBERT+ptrn IO & 96.6 &   95.9 & 96.3 & 97.0 &       95.9 &  97.3 \\
    &    & CmBERT+ptrn IOB2 & 96.0 &   95.2 & 95.5 & 96.6 &       95.3 &  95.3 \\
\cline{2-9}
    & \multirow{4}{*}{M2} & CmBERT IO & 96.2 &   96.2 & 95.6 & 96.9 &       96.2 &  96.7 \\
    &    & CmBERT IOB2 & 96.0 &   96.0 & 95.3 & 96.8 &       96.0 &  96.7 \\
    &    & CmBERT+ptrn IO & 96.3 &   96.1 & 96.0 & 96.7 &       96.1 &  96.9 \\
    &    & CmBERT+ptrn IOB2 & 96.1 &   96.0

All      L1&L2         L1  \
Dataset   Approach Model and tags                                      
Reference M1       CmBERT IO         96.460054  95.699654  96.040045   
                   CmBERT IOB2       96.217547  95.614767  95.789203   
                   CmBERT+ptrn IO    96.620776  95.918409  96.317256   
                   CmBERT+ptrn IOB2  96.012234  95.212618  95.528058   
          M2       CmBERT IO         96.188573  96.157909  95.606135   
                   CmBERT IOB2       95.982694  96.030419  95.291741   
                   CmBERT+ptrn IO    96.289239  96.125104  95.970882   
                   CmBERT+ptrn IOB2  96.077814  96.035353  95.846368   
          M3       CmBERT IO         96.253651  96.122610  95.744014   
                   CmBERT IOB2       96.117493  96.139536  95.621790   
                   CmBERT+ptrn IO    95.845413  95.752708  95.188101   
                   CmBERT+ptrn IOB2  96.336005  96.283451  96.024833   
OCR       M1       CmBERT IO         93.758518  93.355103  93.115639   
                   CmBERT IOB2       93.503830  92.922154  93.109544   
                   CmBERT+ptrn IO    94.318797  93.835619  94.133931   
                   CmBERT+ptrn IOB2  94.085092  93.496475  93.743182   
          M2       CmBERT IO         93.808923  94.128114  93.348428   
                   CmBERT IOB2       93.771105  94.161059  93.195417   
                   CmBERT+ptrn IO    93.882361  94.089300  93.437655   
                   CmBERT+ptrn IOB2  93.722318  94.119129  93.064214   
          M3       CmBERT IO         94.109835  94.412390  93.520808   
                   CmBERT IOB2       93.518969  93.907747  92.901022   
                   CmBERT+ptrn IO    94.111771  94.398637  93.579720   
                   CmBERT+ptrn IOB2  93.660205  94.038940  93.075170   

                                            L2  P-L1+P-L2       Flat  
Dataset   Approach Model and tags                                     
Reference M1       CmBERT IO         96.983179  95.699654  97.042905  
                   CmBERT IOB2       96.751012  95.663863  96.135266  
                   CmBERT+ptrn IO    96.999829  95.918409  97.268102  
                   CmBERT+ptrn IOB2  96.612880  95.331939  95.250599  
          M2       CmBERT IO         96.914893  96.157909  96.703881  
                   CmBERT IOB2       96.839731  96.035288  96.737652  
                   CmBERT+ptrn IO    96.682303  96.125104  96.877739  
                   CmBERT+ptrn IOB2  96.363276  96.068198  96.912063  
          M3       CmBERT IO         96.882258  96.122610  96.736881  
                   CmBERT IOB2       96.729561  96.144481  96.802549  
                   CmBERT+ptrn IO    96.663223  95.752708  96.411974  
                   CmBERT+ptrn IOB2  96.720151  96.254617  97.017190  
OCR       M1       CmBERT IO         94.562345  93.355103  94.158076  
                   CmBERT IOB2       93.993053  93.127077  92.710741  
                   CmBERT+ptrn IO    94.548789  93.835619  94.419470  
                   CmBERT+ptrn IOB2  94.510817  93.708233  94.545170  
          M2       CmBERT IO         94.380526  94.128114  94.534562  
                   CmBERT IOB2       94.486534  94.281452  94.690020  
                   CmBERT+ptrn IO    94.432368  94.089300  94.612859  
                   CmBERT+ptrn IOB2  94.535459  94.189938  94.847359  
          M3       CmBERT IO         94.838207  94.412390  94.773225  
                   CmBERT IOB2       94.285655  93.990849  94.583356  
                   CmBERT+ptrn IO    94.769611  94.398637  94.914480  
                   CmBERT+ptrn IOB2  94.385660  94.189544  94.808493

# 232 - Experiments 1 & 2: tables by classes

In [28]:
import pandas as pd
import json
from pandas import json_normalize

classes = ['eval_PER','eval_ACT','eval_ACT_L1','eval_ACT_L2','eval_DESC','eval_TITREH','eval_TITREP','eval_SPAT','eval_LOC','eval_CARDINAL','eval_FT'
 #'eval_TITRE'
]

def compile_metrics_by_classes(path, classes): 
    rundirs = [f for f in os.listdir(path)]

    df = pd.DataFrame()
    for run_dir in rundirs:
        if 'run' in run_dir:
            run_path = path / run_dir
            nrun = re.search("\d+",run_dir)[0]

            files = [f for f in os.listdir(run_path) if "test_" in f and 'json' in f]
            sizes = [int(re.search("\d+",f)[0]) for f in files]
                
            for file, size in zip(files,sizes):
                file_path = run_path / file
                dftmp = pd.read_json(file_path)
                classes_dict = {key: dftmp[key] for key in classes}
                dftmp = pd.DataFrame.from_dict(classes_dict)
                dftmp = dftmp.T
                dftmp['number'] = dftmp['number'].astype(int)
                dftmp["trainsize"] = size 
                dftmp["run"] = nrun
                dftmp["trainsize_p"] = round(100 * size / MAX_TRAINSET_SIZE, 1)
                df = pd.concat([df, dftmp])
                df["classe"] = df.index
                
    return df.groupby(["run","classe"]).first()

In [29]:
def formatbyClasses(df,classes,metric_name):
    m = df[[metric_name]].reset_index(2)
    m = m.pivot(columns='classe')
    classes = [classe + '-' + metric_name for classe in classes]
    m.columns = classes
    return m

def byClassesDf(metrics_raw_classes,classes):
    precision = formatbyClasses(metrics_raw_classes,classes,'precision')
    recall = formatbyClasses(metrics_raw_classes,classes,'recall')
    f1 = formatbyClasses(metrics_raw_classes,classes,'f1')
    number = formatbyClasses(metrics_raw_classes,classes,'number')
    tmp = precision.join(recall)
    tmp = tmp.join(f1)
    tmp = tmp.join(number)
    return tmp

### Method 1

In [30]:
METRICS_DIR_OCR_M1 = OUT_BASE / "method_1/m1-110-experiment_1_metrics"

In [31]:
# Load models metrics from metrics jsons
camembert_io_ref_m1 = compile_metrics_by_classes(METRICS_DIR_REF_M1 / "111-camembert-ner-multihead-io", classes)
camembert_iob2_ref_m1 = compile_metrics_by_classes(METRICS_DIR_REF_M1 / "112-camembert-ner-multihead-iob2", classes)
prtn_camembert_io_ref_m1 = compile_metrics_by_classes(METRICS_DIR_REF_M1 / "113-pretrained-camembert-ner-multihead-io",classes)
prtn_camembert_iob2_ref_m1 = compile_metrics_by_classes(METRICS_DIR_OCR_M1 / "114-pretrained-camembert-multihead-iob2", classes)

In [32]:
tmp_m1_ref = pd.concat([camembert_io_ref_m1,camembert_iob2_ref_m1,prtn_camembert_io_ref_m1,prtn_camembert_iob2_ref_m1], keys=["CmBERT IO", "CmBERT IOB2", "CmBERT+ptrn IO", "CmBERT+ptrn IOB2"])
metrics_raw_classes_m1_ref = byClassesDf(tmp_m1_ref,classes)
metrics_raw_classes_m1_ref.head()

eval_PER-precision  eval_ACT-precision  eval_ACT_L1-precision  \
          run                                                                  
CmBERT IO 1              0.957364            0.957364               0.000000   
          2              0.965932            0.965932               0.000000   
          3              0.936170            0.960843               0.289474   
          4              0.974200            0.974200               0.000000   
          5              0.929991            0.929991               0.000000   

               eval_ACT_L2-precision  eval_DESC-precision  \
          run                                               
CmBERT IO 1                 0.997714             0.531915   
          2                 0.996571             0.350427   
          3                 0.997713             0.518868   
          4                 0.996568             0.398496   
          5                 0.994866             0.394737   

               eval_TITREH-precision  eval_TITREP-precision  \
          run                                                 
CmBERT IO 1                 1.000000               0.979213   
          2                 0.333333               0.968872   
          3                 0.750000               0.971604   
          4                 0.000000               0.968245   
          5                 0.000000               0.970539   

               eval_SPAT-precision  eval_LOC-precision  \
          run                                            
CmBERT IO 1               0.975768            0.980549   
          2               0.975148            0.973295   
          3               0.969213            0.977143   
          4               0.972206            0.976518   
          5               0.972271            0.977803   

               eval_CARDINAL-precision  ...  eval_ACT-number  \
          run                           ...                    
CmBERT IO 1                   0.977273  ...             1031   
          2                   0.976190  ...             1031   
          3                   0.977273  ...             1031   
          4                   0.977273  ...             1031   
          5                   0.976744  ...             1031   

               eval_ACT_L1-number  eval_ACT_L2-number  eval_DESC-number  \
          run                                                             
CmBERT IO 1                    63                1751                79   
          2                    63                1751                79   
          3                    63                1751                79   
          4                    63                1751                79   
          5                    63                1751                79   

               eval_TITREH-number  eval_TITREP-number  eval_SPAT-number  \
          run                                                             
CmBERT IO 1                    14                1787              1685   
          2                    14                1787              1685   
          3                    14                1787              1685   
          4                    14                1787              1685   
          5                    14                1787              1685   

               eval_LOC-number  eval_CARDINAL-number  eval_FT-number  
          run                                                         
CmBERT IO 1               1747                    43              33  
          2               1747                    43              33  
          3               1747                    43              33  
          4               1747                    43              33  
          5               1747                    43              33  

[5 rows x 44 columns]

In [33]:
METRICS_DIR_OCR_M1 = OUT_BASE / "method_1/m1-120-experiment_2_metrics"

In [34]:
# Load models metrics from metrics jsons
camembert_io_pero_m1 = compile_metrics_by_classes(METRICS_DIR_OCR_M1 / "121-camembert-ner-multihead-io", classes)
camembert_iob2_pero_m1 = compile_metrics_by_classes(METRICS_DIR_OCR_M1 / "122-camembert-ner-multihead-iob2", classes)
prtn_camembert_io_pero_m1 = compile_metrics_by_classes(METRICS_DIR_OCR_M1 / "123-pretrained-camembert-ner-multihead-io",classes)
prtn_camembert_iob2_pero_m1 = compile_metrics_by_classes(METRICS_DIR_OCR_M1 / "124-pretrained-camembert-multihead-iob2", classes)

In [35]:
tmp_m1_pero = pd.concat([camembert_io_pero_m1,camembert_iob2_pero_m1,prtn_camembert_io_pero_m1,prtn_camembert_iob2_pero_m1], keys=["CmBERT IO", "CmBERT IOB2", "CmBERT+ptrn IO", "CmBERT+ptrn IOB2"])
metrics_raw_classes_m1_pero = byClassesDf(tmp_m1_pero,classes)
metrics_raw_classes_m1_pero.head()

eval_PER-precision  eval_ACT-precision  eval_ACT_L1-precision  \
          run                                                                  
CmBERT IO 1              0.927746            0.927746               0.000000   
          2              0.897412            0.900749               0.642857   
          3              0.917222            0.925676               0.333333   
          4              0.899190            0.914179               0.487179   
          5              0.917614            0.917614               0.000000   

               eval_ACT_L2-precision  eval_DESC-precision  \
          run                                               
CmBERT IO 1                 0.980505             0.472727   
          2                 0.982798             0.493671   
          3                 0.983381             0.373913   
          4                 0.981662             0.414286   
          5                 0.981056             0.556818   

               eval_TITREH-precision  eval_TITREP-precision  \
          run                                                 
CmBERT IO 1                 0.666667               0.941566   
          2                 0.750000               0.937327   
          3                 0.166667               0.933443   
          4                 0.666667               0.947164   
          5                 0.500000               0.935644   

               eval_SPAT-precision  eval_LOC-precision  \
          run                                            
CmBERT IO 1               0.941176            0.943814   
          2               0.928697            0.938810   
          3               0.934349            0.940442   
          4               0.947181            0.929174   
          5               0.941211            0.943182   

               eval_CARDINAL-precision  ...  eval_ACT-number  \
          run                           ...                    
CmBERT IO 1                   0.690476  ...             1031   
          2                   0.828571  ...             1031   
          3                   0.622222  ...             1031   
          4                   0.736842  ...             1031   
          5                   0.750000  ...             1031   

               eval_ACT_L1-number  eval_ACT_L2-number  eval_DESC-number  \
          run                                                             
CmBERT IO 1                    63                1751                79   
          2                    63                1751                79   
          3                    63                1751                79   
          4                    63                1751                79   
          5                    63                1751                79   

               eval_TITREH-number  eval_TITREP-number  eval_SPAT-number  \
          run                                                             
CmBERT IO 1                    14                1787              1685   
          2                    14                1787              1685   
          3                    14                1787              1685   
          4                    14                1787              1685   
          5                    14                1787              1685   

               eval_LOC-number  eval_CARDINAL-number  eval_FT-number  
          run                                                         
CmBERT IO 1               1747                    43              32  
          2               1747                    43              32  
          3               1747                    43              32  
          4               1747                    43              32  
          5               1747                    43              32  

[5 rows x 44 columns]

### Method 2

In [36]:
METRICS_DIR_REF_M2 = OUT_BASE / "method_2/m2-210-experiment_1_metrics"

In [37]:
# Load models metrics from metrics jsons
res_classes_m2_ref = []
keys_classes_m2_ref = []

if os.path.exists(METRICS_DIR_REF_M2 / "211-camembert-ner-joint-labelling-io/run_2"):
    camembert_ner_io = compile_metrics_by_classes(METRICS_DIR_REF_M2 / "211-camembert-ner-joint-labelling-io",classes)
    res_classes_m2_ref.append(camembert_ner_io)
    keys_classes_m2_ref.append("CmBERT IO")
    
if os.path.exists(METRICS_DIR_REF_M2 / "212-camembert-ner-joint-labelling-iob2/run_2"):
    camembert_ner_iob2 = compile_metrics_by_classes(METRICS_DIR_REF_M2 / "212-camembert-ner-joint-labelling-iob2",classes)
    res_classes_m2_ref.append(camembert_ner_iob2)
    keys_classes_m2_ref.append("CmBERT IOB2")
    
if os.path.exists(METRICS_DIR_REF_M2 / "213-pretrained-camembert-ner-joint-labelling-io/run_2"):
    ptrn_camembert_ner_io = compile_metrics_by_classes(METRICS_DIR_REF_M2 / "213-pretrained-camembert-ner-joint-labelling-io",classes)
    res_classes_m2_ref.append(ptrn_camembert_ner_io)
    keys_classes_m2_ref.append("CmBERT+ptrn IO")
    
if os.path.exists(METRICS_DIR_REF_M2 / "214-pretrained-camembert-ner-joint-labelling-iob2/run_2"):
    ptrn_camembert_ner_iob2 = compile_metrics_by_classes(METRICS_DIR_REF_M2 / "214-pretrained-camembert-ner-joint-labelling-iob2",classes)
    res_classes_m2_ref.append(ptrn_camembert_ner_iob2)
    keys_classes_m2_ref.append("CmBERT+ptrn IOB2")
    
tmp_m2_ref = pd.concat(res_classes_m2_ref,keys=keys_classes_m2_ref,names=["Test"])
metrics_raw_classes_m2_ref = byClassesDf(tmp_m2_ref,classes)
metrics_raw_classes_m2_ref

eval_PER-precision  eval_ACT-precision  \
Test             run                                           
CmBERT IO        1              0.894783            0.953488   
                 2              0.893766            0.945140   
                 3              0.901596            0.958090   
                 4              0.896269            0.896269   
                 5              0.882405            0.943359   
CmBERT IOB2      1              0.877408            0.917680   
                 2              0.903863            0.950725   
                 3              0.904293            0.938931   
                 4              0.893406            0.912512   
                 5              0.880847            0.916117   
CmBERT+ptrn IO   1              0.875439            0.942663   
                 2              0.897617            0.954851   
                 3              0.885389            0.928705   
                 4              0.858733            0.933206   
                 5              0.897230            0.941233   
CmBERT+ptrn IOB2 1              0.895722            0.944015   
                 2              0.881533            0.913523   
                 3              0.867941            0.946289   
                 4              0.880973            0.945140   
                 5              0.880870            0.941176   

                      eval_ACT_L1-precision  eval_ACT_L2-precision  \
Test             run                                                 
CmBERT IO        1                 0.282828               0.997719   
                 2                 0.360000               0.997151   
                 3                 0.333333               0.999429   
                 4                 0.000000               0.996005   
                 5                 0.299065               0.997713   
CmBERT IOB2      1                 0.287671               0.996011   
                 2                 0.282051               0.996581   
                 3                 0.385714               0.995442   
                 4                 0.431818               0.997141   
                 5                 0.361111               0.995444   
CmBERT+ptrn IO   1                 0.252252               0.993139   
                 2                 0.250000               0.996014   
                 3                 0.285714               0.997719   
                 4                 0.208333               0.996583   
                 5                 0.333333               0.997151   
CmBERT+ptrn IOB2 1                 0.313953               0.996011   
                 2                 0.311475               0.994872   
                 3                 0.236220               0.996579   
                 4                 0.285714               0.994308   
                 5                 0.327434               0.995436   

                      eval_DESC-precision  eval_TITREH-precision  \
Test             run                                               
CmBERT IO        1               0.524752               0.666667   
                 2               0.485437               0.388889   
                 3               0.520000               0.750000   
                 4               0.129412               0.000000   
                 5               0.408000               1.000000   
CmBERT IOB2      1               0.261905               1.000000   
                 2               0.450000               0.500000   
                 3               0.314815               0.500000   
                 4               0.262626               0.571429   
                 5               0.297521               0.400000   
CmBERT+ptrn IO   1               0.351351               0.666667   
                 2               0.495146               0.700000   
                 3               0.319328               0.777778   
                 4               0.349593           

In [38]:
METRICS_DIR_PERO_M2 = OUT_BASE / "method_2/m2-220-experiment_2_metrics"

In [39]:
# Load models metrics from metrics jsons
res_classes_m2_pero = []
keys_classes_m2_pero = []

if os.path.exists(METRICS_DIR_PERO_M2 / "221-camembert-ner-joint-labelling-io/run_2"):
    camembert_ner_io = compile_metrics_by_classes(METRICS_DIR_PERO_M2 / "221-camembert-ner-joint-labelling-io",classes)
    res_classes_m2_pero.append(camembert_ner_io)
    keys_classes_m2_pero.append("CmBERT IO")
    
if os.path.exists(METRICS_DIR_PERO_M2 / "222-camembert-ner-joint-labelling-iob2/run_2"):
    camembert_ner_iob2 = compile_metrics_by_classes(METRICS_DIR_PERO_M2 / "222-camembert-ner-joint-labelling-iob2",classes)
    res_classes_m2_pero.append(camembert_ner_iob2)
    keys_classes_m2_pero.append("CmBERT IOB2")
    
if os.path.exists(METRICS_DIR_PERO_M2 / "223-pretrained-camembert-ner-joint-labelling-io/run_2"):
    ptrn_camembert_ner_io = compile_metrics_by_classes(METRICS_DIR_PERO_M2 / "223-pretrained-camembert-ner-joint-labelling-io",classes)
    res_classes_m2_pero.append(ptrn_camembert_ner_io)
    keys_classes_m2_pero.append("CmBERT+ptrn IO")
    
if os.path.exists(METRICS_DIR_PERO_M2 / "224-pretrained-camembert-ner-joint-labelling-iob2/run_2"):
    ptrn_camembert_ner_iob2 = compile_metrics_by_classes(METRICS_DIR_PERO_M2 / "224-pretrained-camembert-ner-joint-labelling-iob2",classes)
    res_classes_m2_pero.append(ptrn_camembert_ner_iob2)
    keys_classes_m2_pero.append("CmBERT+ptrn IOB2")
    
tmp_m2_pero = pd.concat(res_classes_m2_pero,keys=keys_classes_m2_pero,names=["Test"])
metrics_raw_classes_m2_pero = byClassesDf(tmp_m2_pero,classes)
metrics_raw_classes_m2_pero

eval_PER-precision  eval_ACT-precision  \
Test             run                                           
CmBERT IO        1              0.888496            0.923004   
                 2              0.876869            0.906827   
                 3              0.881786            0.907579   
                 4              0.871366            0.922115   
                 5              0.864494            0.917834   
CmBERT IOB2      1              0.875327            0.918791   
                 2              0.871930            0.919924   
                 3              0.876882            0.939275   
                 4              0.872285            0.912067   
                 5              0.853575            0.879170   
CmBERT+ptrn IO   1              0.871930            0.926457   
                 2              0.862434            0.927805   
                 3              0.873362            0.928367   
                 4              0.881223            0.927563   
                 5              0.869604            0.939275   
CmBERT+ptrn IOB2 1              0.864818            0.910630   
                 2              0.860748            0.905431   
                 3              0.845550            0.907425   
                 4              0.859756            0.928641   
                 5              0.842640            0.884650   

                      eval_ACT_L1-precision  eval_ACT_L2-precision  \
Test             run                                                 
CmBERT IO        1                 0.423077               0.983447   
                 2                 0.264151               0.980560   
                 3                 0.416667               0.981662   
                 4                 0.315789               0.978797   
                 5                 0.263158               0.987861   
CmBERT IOB2      1                 0.352273               0.984465   
                 2                 0.318681               0.980594   
                 3                 0.287037               0.978519   
                 4                 0.353659               0.986262   
                 5                 0.307692               0.989619   
CmBERT+ptrn IO   1                 0.258065               0.985690   
                 2                 0.247706               0.987364   
                 3                 0.285714               0.980605   
                 4                 0.280488               0.985057   
                 5                 0.245614               0.980023   
CmBERT+ptrn IOB2 1                 0.329670               0.990773   
                 2                 0.271605               0.980159   
                 3                 0.256881               0.976836   
                 4                 0.296000               0.989106   
                 5                 0.271605               0.979001   

                      eval_DESC-precision  eval_TITREH-precision  \
Test             run                                               
CmBERT IO        1               0.479167               0.473684   
                 2               0.352941               0.416667   
                 3               0.468750               0.444444   
                 4               0.381679               0.500000   
                 5               0.418182               0.615385   
CmBERT IOB2      1               0.452830               0.416667   
                 2               0.491071               0.750000   
                 3               0.391608               0.454545   
                 4               0.522222               0.466667   
                 5               0.272727               0.714286   
CmBERT+ptrn IO   1               0.377049               0.428571   
                 2               0.366906               0.615385   
                 3               0.419355               0.538462   
                 4               0.382114           

### Method 3

In [40]:
METRICS_DIR_REF_M3 = OUT_BASE / "method_3/m3-310-experiment_1_metrics"

In [41]:
# Load models metrics from metrics jsons
res_classes_m3_ref = []
keys_classes_m3_ref = []

if os.path.exists(METRICS_DIR_REF_M3 / "311-camembert-ner-hierarchical-loss-io/run_2"):
    camembert_ner_io = compile_metrics_by_classes(METRICS_DIR_REF_M3 / "311-camembert-ner-hierarchical-loss-io",classes)
    res_classes_m3_ref.append(camembert_ner_io)
    keys_classes_m3_ref.append("CmBERT IO")
    
if os.path.exists(METRICS_DIR_REF_M3 / "312-camembert-ner-hierarchical-loss-iob2/run_2"):
    camembert_ner_iob2 = compile_metrics_by_classes(METRICS_DIR_REF_M3 / "312-camembert-ner-hierarchical-loss-iob2",classes)
    res_classes_m3_ref.append(camembert_ner_iob2)
    keys_classes_m3_ref.append("CmBERT IOB2")
    
if os.path.exists(METRICS_DIR_REF_M3 / "313-pretrained-camembert-ner-hierarchical-loss-io/run_2"):
    ptrn_camembert_ner_io = compile_metrics_by_classes(METRICS_DIR_REF_M3 / "313-pretrained-camembert-ner-hierarchical-loss-io",classes)
    res_classes_m3_ref.append(ptrn_camembert_ner_io)
    keys_classes_m3_ref.append("CmBERT+ptrn IO")
    
if os.path.exists(METRICS_DIR_REF_M3 / "314-pretrained-camembert-ner-hierarchical-loss-iob2/run_2"):
    ptrn_camembert_ner_iob2 = compile_metrics_by_classes(METRICS_DIR_REF_M3 / "314-pretrained-camembert-ner-hierarchical-loss-iob2",classes)
    res_classes_m3_ref.append(ptrn_camembert_ner_iob2)
    keys_classes_m3_ref.append("CmBERT+ptrn IOB2")
    
tmp_m3_ref = pd.concat(res_classes_m3_ref,keys=keys_classes_m3_ref,names=["Test"])
metrics_raw_classes_m3_ref = byClassesDf(tmp_m3_ref,classes)
metrics_raw_classes_m3_ref

eval_PER-precision  eval_ACT-precision  \
Test             run                                           
CmBERT IO        1              0.870497            0.915974   
                 2              0.888193            0.935610   
                 3              0.892889            0.972421   
                 4              0.914027            0.948544   
                 5              0.899556            0.969185   
CmBERT IOB2      1              0.891228            0.941627   
                 2              0.888594            0.944177   
                 3              0.915468            0.952611   
                 4              0.893178            0.931731   
                 5              0.899274            0.921979   
CmBERT+ptrn IO   1              0.903080            0.966303   
                 2              0.893369            0.893369   
                 3              0.879025            0.920188   
                 4              0.885231            0.945472   
                 5              0.863986            0.924086   
CmBERT+ptrn IOB2 1              0.881890            0.928843   
                 2              0.867017            0.939571   
                 3              0.877515            0.932252   
                 4              0.889860            0.942803   
                 5              0.912156            0.955024   

                      eval_ACT_L1-precision  eval_ACT_L2-precision  \
Test             run                                                 
CmBERT IO        1                 0.277108               0.997146   
                 2                 0.365591               0.996581   
                 3                 0.303030               0.997146   
                 4                 0.440000               0.999429   
                 5                 0.310924               0.998286   
CmBERT IOB2      1                 0.336842               0.996577   
                 2                 0.260870               0.994875   
                 3                 0.423077               0.995447   
                 4                 0.351351               0.994875   
                 5                 0.431373               0.996007   
CmBERT+ptrn IO   1                 0.231579               0.996009   
                 2                 0.000000               0.994875   
                 3                 0.357143               0.996014   
                 4                 0.247423               0.996011   
                 5                 0.235294               0.996014   
CmBERT+ptrn IOB2 1                 0.325843               0.994878   
                 2                 0.230769               0.995442   
                 3                 0.273684               0.994875   
                 4                 0.305263               0.997718   
                 5                 0.365854               0.996577   

                      eval_DESC-precision  eval_TITREH-precision  \
Test             run                                               
CmBERT IO        1               0.321739               0.000000   
                 2               0.452830               0.500000   
                 3               0.444444               0.000000   
                 4               0.489583               0.642857   
                 5               0.456693               0.800000   
CmBERT IOB2      1               0.350000               0.461538   
                 2               0.484211               0.545455   
                 3               0.431373               0.304348   
                 4               0.476190               0.500000   
                 5               0.268519               0.600000   
CmBERT+ptrn IO   1               0.390909               0.600000   
                 2               0.040000               0.000000   
                 3               0.372727               0.370370   
                 4               0.418605           

In [42]:
METRICS_DIR_PERO_M3 = OUT_BASE / "method_3/m3-320-experiment_2_metrics"

In [43]:
# Load models metrics from metrics jsons
res_classes_m3_pero = []
keys_classes_m3_pero = []

if os.path.exists(METRICS_DIR_PERO_M3 / "321-camembert-ner-hierarchical-loss-io/run_2"):
    camembert_ner_io = compile_metrics_by_classes(METRICS_DIR_PERO_M3 / "321-camembert-ner-hierarchical-loss-io",classes)
    res_classes_m3_pero.append(camembert_ner_io)
    keys_classes_m3_pero.append("CmBERT IO")
    
if os.path.exists(METRICS_DIR_PERO_M3 / "322-camembert-ner-hierarchical-loss-iob2/run_2"):
    camembert_ner_iob2 = compile_metrics_by_classes(METRICS_DIR_PERO_M3 / "322-camembert-ner-hierarchical-loss-iob2",classes)
    res_classes_m3_pero.append(camembert_ner_iob2)
    keys_classes_m3_pero.append("CmBERT IOB2")
    
if os.path.exists(METRICS_DIR_PERO_M3 / "323-pretrained-camembert-ner-hierarchical-loss-io/run_2"):
    ptrn_camembert_ner_io = compile_metrics_by_classes(METRICS_DIR_PERO_M3 / "323-pretrained-camembert-ner-hierarchical-loss-io",classes)
    res_classes_m3_pero.append(ptrn_camembert_ner_io)
    keys_classes_m3_pero.append("CmBERT+ptrn IO")
    
if os.path.exists(METRICS_DIR_PERO_M3 / "324-pretrained-camembert-ner-hierarchical-loss-iob2/run_2"):
    ptrn_camembert_ner_iob2 = compile_metrics_by_classes(METRICS_DIR_PERO_M3 / "324-pretrained-camembert-ner-hierarchical-loss-iob2",classes)
    res_classes_m3_pero.append(ptrn_camembert_ner_iob2)
    keys_classes_m3_pero.append("CmBERT+ptrn IOB2")
    
tmp_m3_pero = pd.concat(res_classes_m3_pero,keys=keys_classes_m3_pero,names=["Test"])
metrics_raw_classes_m3_pero = byClassesDf(tmp_m3_pero,classes)
metrics_raw_classes_m3_pero

eval_PER-precision  eval_ACT-precision  \
Test             run                                           
CmBERT IO        1              0.892889            0.926782   
                 2              0.848847            0.903955   
                 3              0.898917            0.934300   
                 4              0.883865            0.922710   
                 5              0.878709            0.926136   
CmBERT IOB2      1              0.886201            0.939036   
                 2              0.879859            0.926597   
                 3              0.812238            0.868324   
                 4              0.875552            0.904986   
                 5              0.857762            0.898242   
CmBERT+ptrn IO   1              0.858261            0.930029   
                 2              0.881988            0.929051   
                 3              0.888688            0.921815   
                 4              0.871391            0.924762   
                 5              0.858648            0.920849   
CmBERT+ptrn IOB2 1              0.848537            0.874545   
                 2              0.847487            0.897267   
                 3              0.847807            0.914692   
                 4              0.836243            0.907993   
                 5              0.860727            0.913826   

                      eval_ACT_L1-precision  eval_ACT_L2-precision  \
Test             run                                                 
CmBERT IO        1                 0.410959               0.986766   
                 2                 0.311927               0.983945   
                 3                 0.397260               0.982857   
                 4                 0.375000               0.981132   
                 5                 0.322222               0.987349   
CmBERT IOB2      1                 0.343434               0.982346   
                 2                 0.289157               0.978929   
                 3                 0.242991               0.985092   
                 4                 0.428571               0.984009   
                 5                 0.250000               0.982877   
CmBERT+ptrn IO   1                 0.247934               0.980594   
                 2                 0.297619               0.982255   
                 3                 0.391304               0.986774   
                 4                 0.268817               0.991334   
                 5                 0.233010               0.984518   
CmBERT+ptrn IOB2 1                 0.387097               0.987327   
                 2                 0.279570               0.987931   
                 3                 0.194444               0.985690   
                 4                 0.228346               0.982326   
                 5                 0.300000               0.977828   

                      eval_DESC-precision  eval_TITREH-precision  \
Test             run                                               
CmBERT IO        1               0.466667               0.666667   
                 2               0.387097               0.583333   
                 3               0.537634               0.700000   
                 4               0.489362               0.461538   
                 5               0.479592               0.642857   
CmBERT IOB2      1               0.408333               0.388889   
                 2               0.324074               0.454545   
                 3               0.256983               0.500000   
                 4               0.510638               0.600000   
                 5               0.377551               0.400000   
CmBERT+ptrn IO   1               0.368421               0.545455   
                 2               0.407407               0.818182   
                 3               0.361345               0.545455   
                 4               0.442478           

In [44]:
metrics_raw_classes_m2_ref

eval_PER-precision  eval_ACT-precision  \
Test             run                                           
CmBERT IO        1              0.894783            0.953488   
                 2              0.893766            0.945140   
                 3              0.901596            0.958090   
                 4              0.896269            0.896269   
                 5              0.882405            0.943359   
CmBERT IOB2      1              0.877408            0.917680   
                 2              0.903863            0.950725   
                 3              0.904293            0.938931   
                 4              0.893406            0.912512   
                 5              0.880847            0.916117   
CmBERT+ptrn IO   1              0.875439            0.942663   
                 2              0.897617            0.954851   
                 3              0.885389            0.928705   
                 4              0.858733            0.933206   
                 5              0.897230            0.941233   
CmBERT+ptrn IOB2 1              0.895722            0.944015   
                 2              0.881533            0.913523   
                 3              0.867941            0.946289   
                 4              0.880973            0.945140   
                 5              0.880870            0.941176   

                      eval_ACT_L1-precision  eval_ACT_L2-precision  \
Test             run                                                 
CmBERT IO        1                 0.282828               0.997719   
                 2                 0.360000               0.997151   
                 3                 0.333333               0.999429   
                 4                 0.000000               0.996005   
                 5                 0.299065               0.997713   
CmBERT IOB2      1                 0.287671               0.996011   
                 2                 0.282051               0.996581   
                 3                 0.385714               0.995442   
                 4                 0.431818               0.997141   
                 5                 0.361111               0.995444   
CmBERT+ptrn IO   1                 0.252252               0.993139   
                 2                 0.250000               0.996014   
                 3                 0.285714               0.997719   
                 4                 0.208333               0.996583   
                 5                 0.333333               0.997151   
CmBERT+ptrn IOB2 1                 0.313953               0.996011   
                 2                 0.311475               0.994872   
                 3                 0.236220               0.996579   
                 4                 0.285714               0.994308   
                 5                 0.327434               0.995436   

                      eval_DESC-precision  eval_TITREH-precision  \
Test             run                                               
CmBERT IO        1               0.524752               0.666667   
                 2               0.485437               0.388889   
                 3               0.520000               0.750000   
                 4               0.129412               0.000000   
                 5               0.408000               1.000000   
CmBERT IOB2      1               0.261905               1.000000   
                 2               0.450000               0.500000   
                 3               0.314815               0.500000   
                 4               0.262626               0.571429   
                 5               0.297521               0.400000   
CmBERT+ptrn IO   1               0.351351               0.666667   
                 2               0.495146               0.700000   
                 3               0.319328               0.777778   
                 4               0.349593           

### Conclusion

In [45]:
ref_classes = [metrics_raw_classes_m1_ref,metrics_raw_classes_m2_ref,metrics_raw_classes_m3_ref]
keys_ref = ["M1","M2","M3"]
metrics_raw_classes_ref = pd.concat(ref_classes, keys=keys_ref,names=["Approach"])

ocr_classes = [metrics_raw_classes_m1_pero,metrics_raw_classes_m2_pero,metrics_raw_classes_m3_pero]
keys_ocr = ["M1","M2","M3"]
metrics_raw_classes_ocr = pd.concat(ocr_classes, keys=keys_ocr,names=["Approach"])

datasets = [metrics_raw_classes_ref,metrics_raw_classes_ocr]
keys = ["Reference","OCR"]
metrics_raw_classes = pd.concat(datasets, keys=keys,names=["Dataset"])
metrics_raw_classes

eval_PER-precision  \
Dataset   Approach                  run                       
Reference M1       CmBERT IO        1              0.957364   
                                    2              0.965932   
                                    3              0.936170   
                                    4              0.974200   
                                    5              0.929991   
...                                                     ...   
OCR       M3       CmBERT+ptrn IOB2 1              0.848537   
                                    2              0.847487   
                                    3              0.847807   
                                    4              0.836243   
                                    5              0.860727   

                                         eval_ACT-precision  \
Dataset   Approach                  run                       
Reference M1       CmBERT IO        1              0.957364   
                                    2              0.965932   
                                    3              0.960843   
                                    4              0.974200   
                                    5              0.929991   
...                                                     ...   
OCR       M3       CmBERT+ptrn IOB2 1              0.874545   
                                    2              0.897267   
                                    3              0.914692   
                                    4              0.907993   
                                    5              0.913826   

                                         eval_ACT_L1-precision  \
Dataset   Approach                  run                          
Reference M1       CmBERT IO        1                 0.000000   
                                    2                 0.000000   
                                    3                 0.289474   
                                    4                 0.000000   
                                    5                 0.000000   
...                                                        ...   
OCR       M3       CmBERT+ptrn IOB2 1                 0.387097   
                                    2                 0.279570   
                                    3                 0.194444   
                                    4                 0.228346   
                                    5                 0.300000   

                                         eval_ACT_L2-precision  \
Dataset   Approach                  run                          
Reference M1       CmBERT IO        1                 0.997714   
                                    2                 0.996571   
                                    3                 0.997713   
                                    4                 0.996568   
                                    5                 0.994866   
...                                                        ...   
OCR       M3       CmBERT+ptrn IOB2 1                 0.987327   
                                    2                 0.987931   
                                    3                 0.985690   
                                    4                 0.982326   
                                    5                 0.977828   

                                         eval_DESC-precision  \
Dataset   Approach                  run                        
Reference M1       CmBERT IO        1               0.531915   
                                    2               0.350427   
                                    3               0.518868   
                                    4               0.398496   
                                    5               0.394737   
...                                                      ...   
OCR       M3       CmBERT+ptrn IOB2 1               0.258621   
                                    2               0.285714   
                                    3      

In [46]:
eval_ = []
for elem in metrics_raw_classes.columns:
    if 'number' not in elem and 'f1' in elem:
        eval_.append(elem)
        
metrics_raw_classes = metrics_raw_classes[eval_].groupby(level=(0,1,2)).mean()
metrics_raw_classes[eval_] = metrics_raw_classes[eval_].multiply(100., axis=1)
metrics_raw_classes

eval_PER-f1  eval_ACT-f1  eval_ACT_L1-f1  \
Dataset   Approach                                                              
Reference M1       CmBERT IO           91.799822    94.757473        4.356436   
                   CmBERT IOB2         90.438916    93.986777       24.826957   
                   CmBERT+ptrn IO      92.563930    95.699587       17.848898   
                   CmBERT+ptrn IOB2    89.813091    93.597412       25.666546   
          M2       CmBERT IO           90.644476    94.470645       31.519772   
                   CmBERT IOB2         90.338967    93.783417       35.344414   
                   CmBERT+ptrn IO      90.115511    94.613351       31.622461   
                   CmBERT+ptrn IOB2    90.119513    94.394823       35.416521   
          M3       CmBERT IO           90.531106    94.480437       40.921532   
                   CmBERT IOB2         90.794113    94.328273       38.967055   
                   CmBERT+ptrn IO      89.997110    94.066241       25.551067   
                   CmBERT+ptrn IOB2    90.388034    94.573537       35.831492   
OCR       M1       CmBERT IO           90.043853    92.720677       14.690408   
                   CmBERT IOB2         88.690412    92.817873       19.420856   
                   CmBERT+ptrn IO      89.329012    93.284056       17.466280   
                   CmBERT+ptrn IOB2    88.953135    92.599081       20.538704   
          M2       CmBERT IO           89.535834    93.055330       36.243420   
                   CmBERT IOB2         88.999364    92.669387       36.561067   
                   CmBERT+ptrn IO      88.959022    93.412767       32.061949   
                   CmBERT+ptrn IOB2    87.812343    91.894318       34.398398   
          M3       CmBERT IO           89.572129    93.017104       41.233226   
                   CmBERT IOB2         88.169382    91.928783       35.443941   
                   CmBERT+ptrn IO      88.686693    92.877581       33.699667   
                   CmBERT+ptrn IOB2    87.566320    91.807388       32.726174   

                                     eval_ACT_L2-f1  eval_DESC-f1  \
Dataset   Approach                                                  
Reference M1       CmBERT IO              99.640116     49.561868   
                   CmBERT IOB2            99.708938     43.286669   
                   CmBERT+ptrn IO         99.680245     53.528603   
                   CmBERT+ptrn IOB2       99.697578     44.407316   
          M2       CmBERT IO              99.794455     47.069827   
                   CmBERT IOB2            99.697562     36.810314   
                   CmBERT+ptrn IO         99.697425     47.674517   
                   CmBERT+ptrn IOB2       99.652010     42.875308   
          M3       CmBERT IO              99.805897     51.492058   
                   CmBERT IOB2            99.669142     44.759840   
                   CmBERT+ptrn IO         99.703398     36.070327   
                   CmBERT+ptrn IOB2       99.697669     48.314903   
OCR       M1       CmBERT IO              97.991390     49.266450   
                   CmBERT IOB2            97.996548     42.167680   
                   CmBERT+ptrn IO         97.994742     50.928631   
                   CmBERT+ptrn IOB2       98.246087     48.195187   
          M2       CmBERT IO              98.054476     47.376332   
                   CmBERT IOB2            98.302388     49.178469   
                   CmBERT+ptrn IO         98.250385     48.065263   
                   CmBERT+ptrn IOB2       98.420439     39.167631   
          M3       CmBERT IO              98.243319     52.284171   
                   CmBERT IOB2            98.287141     43.927508   
                   CmBERT+ptrn IO         98.271473     46.319535   
                   CmBERT+ptrn IOB2       98.313692     38.342908   

                                     eval_TITREH-f1  eval_TITREP-f1  \
Dataset   Approach                                              

In [47]:
averaged_classes = metrics_raw_classes.copy()

start_classes = list(averaged_classes.columns)
final_classes = [classe.replace('eval_','') for classe in classes]
columns_names = {start_classes[i]: final_classes[i] for i in range(len(final_classes))}

# Set pretty names
averaged_classes.index.names = ['Dataset','Approach',"Model & tags"]
averaged_classes.rename(columns=columns_names, errors="raise", inplace=True)

classes_f = ['PER','ACT','DESC','TITREH','TITREP','SPAT','LOC','CARDINAL','FT']
averaged_classes = averaged_classes[classes_f]
averaged_classes

PER        ACT       DESC  \
Dataset   Approach Model & tags                                        
Reference M1       CmBERT IO         91.799822  94.757473  49.561868   
                   CmBERT IOB2       90.438916  93.986777  43.286669   
                   CmBERT+ptrn IO    92.563930  95.699587  53.528603   
                   CmBERT+ptrn IOB2  89.813091  93.597412  44.407316   
          M2       CmBERT IO         90.644476  94.470645  47.069827   
                   CmBERT IOB2       90.338967  93.783417  36.810314   
                   CmBERT+ptrn IO    90.115511  94.613351  47.674517   
                   CmBERT+ptrn IOB2  90.119513  94.394823  42.875308   
          M3       CmBERT IO         90.531106  94.480437  51.492058   
                   CmBERT IOB2       90.794113  94.328273  44.759840   
                   CmBERT+ptrn IO    89.997110  94.066241  36.070327   
                   CmBERT+ptrn IOB2  90.388034  94.573537  48.314903   
OCR       M1       CmBERT IO         90.043853  92.720677  49.266450   
                   CmBERT IOB2       88.690412  92.817873  42.167680   
                   CmBERT+ptrn IO    89.329012  93.284056  50.928631   
                   CmBERT+ptrn IOB2  88.953135  92.599081  48.195187   
          M2       CmBERT IO         89.535834  93.055330  47.376332   
                   CmBERT IOB2       88.999364  92.669387  49.178469   
                   CmBERT+ptrn IO    88.959022  93.412767  48.065263   
                   CmBERT+ptrn IOB2  87.812343  91.894318  39.167631   
          M3       CmBERT IO         89.572129  93.017104  52.284171   
                   CmBERT IOB2       88.169382  91.928783  43.927508   
                   CmBERT+ptrn IO    88.686693  92.877581  46.319535   
                   CmBERT+ptrn IOB2  87.566320  91.807388  38.342908   

                                        TITREH     TITREP       SPAT  \
Dataset   Approach Model & tags                                        
Reference M1       CmBERT IO         11.686275  97.352885  97.476534   
                   CmBERT IOB2       22.464858  97.261297  97.674942   
                   CmBERT+ptrn IO    50.462824  97.246492  97.626150   
                   CmBERT+ptrn IOB2  41.462868  97.214125  97.681952   
          M2       CmBERT IO         39.903846  97.495406  97.253573   
                   CmBERT IOB2       39.806076  97.332429  97.350988   
                   CmBERT+ptrn IO    58.148272  97.562956  98.223989   
                   CmBERT+ptrn IOB2  38.634699  97.053840  98.359877   
          M3       CmBERT IO         33.042901  97.554721  97.539117   
                   CmBERT IOB2       40.705579  97.326496  97.345912   
                   CmBERT+ptrn IO    45.113034  97.464507  96.807606   
                   CmBERT+ptrn IOB2  44.818126  97.460048  98.378060   
OCR       M1       CmBERT IO         20.578431  94.507176  94.282584   
                   CmBERT IOB2       27.341270  94.330636  94.781748   
                   CmBERT+ptrn IO    39.739216  95.118416  96.041616   
                   CmBERT+ptrn IOB2  37.788360  94.833236  96.004853   
          M2       CmBERT IO         39.909772  94.686317  94.961686   
                   CmBERT IOB2       45.780381  94.620276  94.979734   
                   CmBERT+ptrn IO    56.984127  94.978049  94.756590   
                   CmBERT+ptrn IOB2  40.770953  94.962686  95.461803   
          M3       CmBERT IO         54.616712  95.051834  94.963670   
                   CmBERT IOB2       38.330460  94.493064  95.120737   
                   CmBERT+ptrn IO    55.600000  95.440169  95.388511   
                   CmBERT+ptrn IOB2  43.726601  94.769604  95.710377   

                                           LOC   CARDINAL         FT  
Dataset   Approach Model & tags                                       
Reference M1       CmBERT IO         97.850920  98.139346  36.176726  
                   CmBERT IOB2       97.555637  96.497072  51.549715  
             

In [48]:
latex_table_classes = averaged_classes.copy()

caption = "F1 score measured for each approach, pre-trained model and tag format (mean of 5 runs) for each entity type."
print(latex_table_classes.to_latex(float_format="%.1f", multirow=True, caption=caption))
latex_table_classes

\begin{table}
\centering
\caption{F1 score measured for each approach, pre-trained model and tag format (mean of 5 runs) for each entity type.}
\begin{tabular}{lllrrrrrrrrr}
\toprule
    &    &                  &  PER &  ACT &  DESC &  TITREH &  TITREP &  SPAT &  LOC &  CARDINAL &   FT \\
Dataset & Approach & Model \& tags &      &      &       &         &         &       &      &           &      \\
\midrule
\multirow{12}{*}{Reference} & \multirow{4}{*}{M1} & CmBERT IO & 91.8 & 94.8 &  49.6 &    11.7 &    97.4 &  97.5 & 97.9 &      98.1 & 36.2 \\
    &    & CmBERT IOB2 & 90.4 & 94.0 &  43.3 &    22.5 &    97.3 &  97.7 & 97.6 &      96.5 & 51.5 \\
    &    & CmBERT+ptrn IO & 92.6 & 95.7 &  53.5 &    50.5 &    97.2 &  97.6 & 97.6 &      98.4 & 53.4 \\
    &    & CmBERT+ptrn IOB2 & 89.8 & 93.6 &  44.4 &    41.5 &    97.2 &  97.7 & 97.4 &      97.2 & 50.4 \\
\cline{2-12}
    & \multirow{4}{*}{M2} & CmBERT IO & 90.6 & 94.5 &  47.1 &    39.9 &    97.5 &  97.3 & 97.2 &      97.5 & 58.7 \\
  

PER        ACT       DESC  \
Dataset   Approach Model & tags                                        
Reference M1       CmBERT IO         91.799822  94.757473  49.561868   
                   CmBERT IOB2       90.438916  93.986777  43.286669   
                   CmBERT+ptrn IO    92.563930  95.699587  53.528603   
                   CmBERT+ptrn IOB2  89.813091  93.597412  44.407316   
          M2       CmBERT IO         90.644476  94.470645  47.069827   
                   CmBERT IOB2       90.338967  93.783417  36.810314   
                   CmBERT+ptrn IO    90.115511  94.613351  47.674517   
                   CmBERT+ptrn IOB2  90.119513  94.394823  42.875308   
          M3       CmBERT IO         90.531106  94.480437  51.492058   
                   CmBERT IOB2       90.794113  94.328273  44.759840   
                   CmBERT+ptrn IO    89.997110  94.066241  36.070327   
                   CmBERT+ptrn IOB2  90.388034  94.573537  48.314903   
OCR       M1       CmBERT IO         90.043853  92.720677  49.266450   
                   CmBERT IOB2       88.690412  92.817873  42.167680   
                   CmBERT+ptrn IO    89.329012  93.284056  50.928631   
                   CmBERT+ptrn IOB2  88.953135  92.599081  48.195187   
          M2       CmBERT IO         89.535834  93.055330  47.376332   
                   CmBERT IOB2       88.999364  92.669387  49.178469   
                   CmBERT+ptrn IO    88.959022  93.412767  48.065263   
                   CmBERT+ptrn IOB2  87.812343  91.894318  39.167631   
          M3       CmBERT IO         89.572129  93.017104  52.284171   
                   CmBERT IOB2       88.169382  91.928783  43.927508   
                   CmBERT+ptrn IO    88.686693  92.877581  46.319535   
                   CmBERT+ptrn IOB2  87.566320  91.807388  38.342908   

                                        TITREH     TITREP       SPAT  \
Dataset   Approach Model & tags                                        
Reference M1       CmBERT IO         11.686275  97.352885  97.476534   
                   CmBERT IOB2       22.464858  97.261297  97.674942   
                   CmBERT+ptrn IO    50.462824  97.246492  97.626150   
                   CmBERT+ptrn IOB2  41.462868  97.214125  97.681952   
          M2       CmBERT IO         39.903846  97.495406  97.253573   
                   CmBERT IOB2       39.806076  97.332429  97.350988   
                   CmBERT+ptrn IO    58.148272  97.562956  98.223989   
                   CmBERT+ptrn IOB2  38.634699  97.053840  98.359877   
          M3       CmBERT IO         33.042901  97.554721  97.539117   
                   CmBERT IOB2       40.705579  97.326496  97.345912   
                   CmBERT+ptrn IO    45.113034  97.464507  96.807606   
                   CmBERT+ptrn IOB2  44.818126  97.460048  98.378060   
OCR       M1       CmBERT IO         20.578431  94.507176  94.282584   
                   CmBERT IOB2       27.341270  94.330636  94.781748   
                   CmBERT+ptrn IO    39.739216  95.118416  96.041616   
                   CmBERT+ptrn IOB2  37.788360  94.833236  96.004853   
          M2       CmBERT IO         39.909772  94.686317  94.961686   
                   CmBERT IOB2       45.780381  94.620276  94.979734   
                   CmBERT+ptrn IO    56.984127  94.978049  94.756590   
                   CmBERT+ptrn IOB2  40.770953  94.962686  95.461803   
          M3       CmBERT IO         54.616712  95.051834  94.963670   
                   CmBERT IOB2       38.330460  94.493064  95.120737   
                   CmBERT+ptrn IO    55.600000  95.440169  95.388511   
                   CmBERT+ptrn IOB2  43.726601  94.769604  95.710377   

                                           LOC   CARDINAL         FT  
Dataset   Approach Model & tags                                       
Reference M1       CmBERT IO         97.850920  98.139346  36.176726  
                   CmBERT IOB2       97.555637  96.497072  51.549715  
             

In [49]:
averaged_classes_ref = averaged_classes.loc[['Reference']]
averaged_classes_ref = averaged_classes_ref.reset_index(0)
del averaged_classes_ref["Dataset"]
latex_table = averaged_classes_ref.copy()
caption = "F1 score measured for each approach, pre-trained model and tag format (mean of 5 runs) on the reference dataset for each entity type."
print(latex_table.to_latex(float_format="%.1f", multirow=True, caption=caption))
latex_table

\begin{table}
\centering
\caption{F1 score measured for each approach, pre-trained model and tag format (mean of 5 runs) on the reference dataset for each entity type.}
\begin{tabular}{llrrrrrrrrr}
\toprule
   &                  &  PER &  ACT &  DESC &  TITREH &  TITREP &  SPAT &  LOC &  CARDINAL &   FT \\
Approach & Model \& tags &      &      &       &         &         &       &      &           &      \\
\midrule
\multirow{4}{*}{M1} & CmBERT IO & 91.8 & 94.8 &  49.6 &    11.7 &    97.4 &  97.5 & 97.9 &      98.1 & 36.2 \\
   & CmBERT IOB2 & 90.4 & 94.0 &  43.3 &    22.5 &    97.3 &  97.7 & 97.6 &      96.5 & 51.5 \\
   & CmBERT+ptrn IO & 92.6 & 95.7 &  53.5 &    50.5 &    97.2 &  97.6 & 97.6 &      98.4 & 53.4 \\
   & CmBERT+ptrn IOB2 & 89.8 & 93.6 &  44.4 &    41.5 &    97.2 &  97.7 & 97.4 &      97.2 & 50.4 \\
\cline{1-11}
\multirow{4}{*}{M2} & CmBERT IO & 90.6 & 94.5 &  47.1 &    39.9 &    97.5 &  97.3 & 97.2 &      97.5 & 58.7 \\
   & CmBERT IOB2 & 90.3 & 93.8 &  36.8 &    39.8

PER        ACT       DESC     TITREH  \
Approach Model & tags                                                   
M1       CmBERT IO         91.799822  94.757473  49.561868  11.686275   
         CmBERT IOB2       90.438916  93.986777  43.286669  22.464858   
         CmBERT+ptrn IO    92.563930  95.699587  53.528603  50.462824   
         CmBERT+ptrn IOB2  89.813091  93.597412  44.407316  41.462868   
M2       CmBERT IO         90.644476  94.470645  47.069827  39.903846   
         CmBERT IOB2       90.338967  93.783417  36.810314  39.806076   
         CmBERT+ptrn IO    90.115511  94.613351  47.674517  58.148272   
         CmBERT+ptrn IOB2  90.119513  94.394823  42.875308  38.634699   
M3       CmBERT IO         90.531106  94.480437  51.492058  33.042901   
         CmBERT IOB2       90.794113  94.328273  44.759840  40.705579   
         CmBERT+ptrn IO    89.997110  94.066241  36.070327  45.113034   
         CmBERT+ptrn IOB2  90.388034  94.573537  48.314903  44.818126   

                              TITREP       SPAT        LOC   CARDINAL  \
Approach Model & tags                                                   
M1       CmBERT IO         97.352885  97.476534  97.850920  98.139346   
         CmBERT IOB2       97.261297  97.674942  97.555637  96.497072   
         CmBERT+ptrn IO    97.246492  97.626150  97.600769  98.380112   
         CmBERT+ptrn IOB2  97.214125  97.681952  97.350078  97.219995   
M2       CmBERT IO         97.495406  97.253573  97.214809  97.460140   
         CmBERT IOB2       97.332429  97.350988  97.377046  97.898580   
         CmBERT+ptrn IO    97.562956  98.223989  97.324784  98.374766   
         CmBERT+ptrn IOB2  97.053840  98.359877  97.339982  97.898769   
M3       CmBERT IO         97.554721  97.539117  97.255276  97.909277   
         CmBERT IOB2       97.326496  97.345912  97.361644  95.768983   
         CmBERT+ptrn IO    97.464507  96.807606  97.387768  96.192415   
         CmBERT+ptrn IOB2  97.460048  98.378060  97.371193  98.134063   

                                  FT  
Approach Model & tags                 
M1       CmBERT IO         36.176726  
         CmBERT IOB2       51.549715  
         CmBERT+ptrn IO    53.423239  
         CmBERT+ptrn IOB2  50.419259  
M2       CmBERT IO         58.715928  
         CmBERT IOB2       55.978269  
         CmBERT+ptrn IO    57.300857  
         CmBERT+ptrn IOB2  62.291779  
M3       CmBERT IO         55.967049  
         CmBERT IOB2       51.786387  
         CmBERT+ptrn IO    42.861735  
         CmBERT+ptrn IOB2  64.133260

In [50]:
averaged_classes_ocr = averaged_classes.loc[['OCR']]
averaged_classes_ocr = averaged_classes_ocr.reset_index(0)
del averaged_classes_ocr["Dataset"]
latex_table = averaged_classes_ocr.copy()
caption = "F1 score measured for each approach, pre-trained model and tag format (mean of 5 runs) on the noisy dataset for each entity type."
print(latex_table.to_latex(float_format="%.1f", multirow=True, caption=caption))
latex_table

\begin{table}
\centering
\caption{F1 score measured for each approach, pre-trained model and tag format (mean of 5 runs) on the noisy dataset for each entity type.}
\begin{tabular}{llrrrrrrrrr}
\toprule
   &                  &  PER &  ACT &  DESC &  TITREH &  TITREP &  SPAT &  LOC &  CARDINAL &   FT \\
Approach & Model \& tags &      &      &       &         &         &       &      &           &      \\
\midrule
\multirow{4}{*}{M1} & CmBERT IO & 90.0 & 92.7 &  49.3 &    20.6 &    94.5 &  94.3 & 94.4 &      69.5 & 48.1 \\
   & CmBERT IOB2 & 88.7 & 92.8 &  42.2 &    27.3 &    94.3 &  94.8 & 94.2 &      79.5 & 33.7 \\
   & CmBERT+ptrn IO & 89.3 & 93.3 &  50.9 &    39.7 &    95.1 &  96.0 & 95.1 &      77.0 & 50.3 \\
   & CmBERT+ptrn IOB2 & 89.0 & 92.6 &  48.2 &    37.8 &    94.8 &  96.0 & 94.8 &      79.4 & 42.3 \\
\cline{1-11}
\multirow{4}{*}{M2} & CmBERT IO & 89.5 & 93.1 &  47.4 &    39.9 &    94.7 &  95.0 & 94.4 &      74.0 & 41.9 \\
   & CmBERT IOB2 & 89.0 & 92.7 &  49.2 &    45.8 &  

PER        ACT       DESC     TITREH  \
Approach Model & tags                                                   
M1       CmBERT IO         90.043853  92.720677  49.266450  20.578431   
         CmBERT IOB2       88.690412  92.817873  42.167680  27.341270   
         CmBERT+ptrn IO    89.329012  93.284056  50.928631  39.739216   
         CmBERT+ptrn IOB2  88.953135  92.599081  48.195187  37.788360   
M2       CmBERT IO         89.535834  93.055330  47.376332  39.909772   
         CmBERT IOB2       88.999364  92.669387  49.178469  45.780381   
         CmBERT+ptrn IO    88.959022  93.412767  48.065263  56.984127   
         CmBERT+ptrn IOB2  87.812343  91.894318  39.167631  40.770953   
M3       CmBERT IO         89.572129  93.017104  52.284171  54.616712   
         CmBERT IOB2       88.169382  91.928783  43.927508  38.330460   
         CmBERT+ptrn IO    88.686693  92.877581  46.319535  55.600000   
         CmBERT+ptrn IOB2  87.566320  91.807388  38.342908  43.726601   

                              TITREP       SPAT        LOC   CARDINAL  \
Approach Model & tags                                                   
M1       CmBERT IO         94.507176  94.282584  94.409681  69.531118   
         CmBERT IOB2       94.330636  94.781748  94.204201  79.458108   
         CmBERT+ptrn IO    95.118416  96.041616  95.096117  77.020229   
         CmBERT+ptrn IOB2  94.833236  96.004853  94.815366  79.441012   
M2       CmBERT IO         94.686317  94.961686  94.357316  73.965405   
         CmBERT IOB2       94.620276  94.979734  94.161251  80.683880   
         CmBERT+ptrn IO    94.978049  94.756590  94.859238  75.554274   
         CmBERT+ptrn IOB2  94.962686  95.461803  94.770036  78.521844   
M3       CmBERT IO         95.051834  94.963670  94.541947  79.888247   
         CmBERT IOB2       94.493064  95.120737  94.148703  76.837110   
         CmBERT+ptrn IO    95.440169  95.388511  94.997698  75.300806   
         CmBERT+ptrn IOB2  94.769604  95.710377  94.553500  82.680821   

                                  FT  
Approach Model & tags                 
M1       CmBERT IO         48.143436  
         CmBERT IOB2       33.662753  
         CmBERT+ptrn IO    50.341841  
         CmBERT+ptrn IOB2  42.294429  
M2       CmBERT IO         41.934837  
         CmBERT IOB2       45.645323  
         CmBERT+ptrn IO    50.707517  
         CmBERT+ptrn IOB2  56.524140  
M3       CmBERT IO         53.876494  
         CmBERT IOB2       42.857044  
         CmBERT+ptrn IO    56.144510  
         CmBERT+ptrn IOB2  52.197355